# PRIMERA FUENTE DE DATOS: World Bank

In [ ]:
# - requests: para obtener datos de Internet.
# - pandas: para organizar los datos en tablas.
# - json: para trabajar con información en formato JSON.
import requests
import pandas as pd
import json

# URL de la API del Banco Mundial donde están los datos de anemia infantil.
base_url = "http://api.worldbank.org/v2/country/ALL/indicator/SH.ANM.CHLD.ZS"

# Indicamos a la API que queremos los datos en formato JSON.
params = {
    "format": "json"
}

# Creamos una lista vacía donde guardaremos los datos.
all_data = []

# Enviamos la solicitud a la API.
response = requests.get(base_url, params=params)

# Verificamos si la solicitud fue exitosa (código 200 significa éxito).
if response.status_code == 200:
    # Convertimos la respuesta de la API en un formato que Python pueda entender (JSON).
    data = response.json()
    
    # Revisamos si hay datos útiles en la respuesta.
    if len(data) > 1:  # Los datos que necesitamos están en la segunda parte de la respuesta.
        all_data = data[1]  # Guardamos esos datos en nuestra lista.
    else:
        print("No se encontraron datos en la respuesta.")
else:
    # Si ocurre un error, mostramos el código de error.
    print("Error al obtener los datos:", response.status_code)

# Ahora, organizamos los datos en una tabla usando pandas.
df_worldbank = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV (una hoja de cálculo).
output_file = "world_bank_anemia.csv"
df_worldbank.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo sin incluir índices.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# SEGUNDA FUENTE: Global Health Observatory

In [ ]:
import requests
import pandas as pd
import json

# URL de la API para datos de nutrición y anemia infantil.
base_url = "https://ghoapi.azureedge.net/api/NUTRITION_ANAEMIA_CHILDREN_NUM"

# Configuramos los parámetros para obtener los datos en partes (paginación):
# - "$top": cuántos registros obtener por solicitud.
# - "$skip": cuántos registros saltar para la siguiente solicitud.
params = {
    "$top": 1000,  # Pedimos 1000 registros por solicitud.
    "$skip": 0     # Empezamos desde el inicio.
}

# Lista vacía para guardar todos los datos.
all_data = []

# Usamos un ciclo para seguir pidiendo datos hasta que no queden más.
while True:
    # Hacemos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Si la solicitud fue exitosa:
    if response.status_code == 200:
        # Convertimos la respuesta en un formato JSON y agregamos los datos a nuestra lista.
        data = response.json()
        all_data.extend(data["value"])  # Extendemos la lista con los nuevos datos.
        
        # Si la cantidad de datos obtenidos es menor que "$top", significa que no hay más datos.
        if len(data["value"]) < params["$top"]:
            break  # Terminamos el ciclo.
        
        # Si hay más datos, incrementamos "$skip" para pedir la siguiente página.
        params["$skip"] += params["$top"]
    else:
        # Si ocurre un error, mostramos el código de error y terminamos el ciclo.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos en una tabla con pandas.
df_anemia = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV.
output_file = "nutrition_anemia_children.csv"
df_anemia.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos el archivo.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que se creó el archivo.

# TERCERA FUENTE: DHS Program

### ENCUESTA 1: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER TIPO DE ANEMIA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de nutrición (anemia en este caso).
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_ANY"

# Parámetros que se enviarán a la API:
# - "perpage": número máximo de registros por solicitud (aquí pedimos 1000).
# - "page": indica el número de la página que estamos solicitando (empezamos en la 1).
params = {
    "perpage": 1000,  # Máximo de registros por página.
    "page": 1         # Comenzamos desde la página 1.
}

# Creamos una lista vacía para almacenar todos los datos obtenidos de la API.
all_data = []

# Usamos un ciclo para descargar todos los datos disponibles en la API.
while True:
    # Realizamos una solicitud a la API con los parámetros actuales.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de respuesta 200).
    if response.status_code == 200:
        # Convertimos la respuesta de la API a formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Extraemos los datos dentro de la clave "Data" y los agregamos a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos:
        # Si la cantidad de datos obtenidos es menor que el límite "perpage",
        # significa que no hay más páginas que consultar.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si aún hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código distinto de 200), mostramos el código de error y terminamos.
        print("Error al obtener los datos:", response.status_code)
        break

# Una vez descargados todos los datos, los organizamos en una tabla con Pandas.
df_any = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_any_anemia.csv".
output_file = "df_any_anemia.csv"
df_any.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo se creó.

### ENCUESTA 2: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA LEVE

In [ ]:
import requests
import pandas as pd
import json
# URL de la API del DHS para obtener datos de anemia infantil leve.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MLD"

# Parámetros que enviamos a la API:
# - "perpage": cantidad máxima de datos que queremos recibir por solicitud (1000 aquí).
# - "page": indica el número de página que solicitamos (iniciamos desde la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Empezamos desde la primera página.
}

# Creamos una lista vacía para guardar todos los datos descargados de la API.
all_data = []

# Usamos un ciclo para realizar solicitudes a la API hasta que descarguemos todos los datos.
while True:
    # Realizamos la solicitud a la API con los parámetros configurados.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código 200 indica éxito).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manejar en Python).
        data = response.json()
        
        # Agregamos los datos obtenidos (clave "Data") a nuestra lista.
        # Usamos "get" para evitar errores si la clave no existe.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si ya no hay más datos disponibles:
        # Si el número de datos recibidos es menor al límite "perpage", hemos llegado al final.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque no hay más datos.
        
        # Si aún hay más datos, pasamos a la siguiente página incrementando el número de página.
        params["page"] += 1
    else:
        # Si ocurre un error (respuesta distinta de 200), mostramos un mensaje con el código de error.
        print("Error al obtener los datos:", response.status_code)
        break

# Cuando terminamos de descargar los datos, los organizamos en una tabla usando pandas.
df_mld = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mld_anemia.csv".
output_file = "df_mld_anemia.csv"
df_mld.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado.

Archivo CSV creado exitosamente: df_mld_anemia.csv


### ENCUESTA 3: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA MODERADA

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS para obtener datos de anemia infantil moderada.
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_MOD"

# Parámetros que enviamos a la API:
# - "perpage": define el número máximo de registros a recibir por solicitud (en este caso, 1000).
# - "page": indica el número de página que se solicita (empezamos en la página 1).
params = {
    "perpage": 1000,  # Pedimos hasta 1000 registros por página.
    "page": 1         # Comenzamos desde la primera página.
}

# Lista vacía para almacenar todos los datos descargados de la API.
all_data = []

# Ciclo para realizar solicitudes repetidas hasta que se descarguen todos los datos disponibles.
while True:
    # Realizamos una solicitud GET a la API utilizando la URL base y los parámetros definidos.
    response = requests.get(base_url, params=params)
    
    # Verificamos si la solicitud fue exitosa (código de estado 200).
    if response.status_code == 200:
        # Convertimos la respuesta en formato JSON (fácil de manipular en Python).
        data = response.json()
        
        # Extraemos los datos de la clave "Data" y los agregamos a la lista "all_data".
        # Usamos "get" para evitar errores si la clave no está presente.
        all_data.extend(data.get("Data", []))
        
        # Verificamos si hemos llegado al final de los datos disponibles:
        # Si el número de registros recibidos es menor que "perpage", significa que no hay más páginas.
        if len(data.get("Data", [])) < params["perpage"]:
            break  # Salimos del ciclo porque ya no hay más datos.
        
        # Si todavía hay más datos, incrementamos el número de página para la siguiente solicitud.
        params["page"] += 1
    else:
        # Si ocurre un error (código diferente de 200), mostramos el código de error y detenemos el proceso.
        print("Error al obtener los datos:", response.status_code)
        break

# Organizamos los datos descargados en una tabla (DataFrame) usando pandas.
df_mod = pd.json_normalize(all_data)

# Guardamos la tabla en un archivo CSV llamado "df_mod_anemia.csv".
output_file = "df_mod_anemia.csv"
df_mod.to_csv(output_file, index=False, encoding="utf-8")  # Guardamos sin incluir índices y en formato UTF-8.
print(f"Archivo CSV creado exitosamente: {output_file}")  # Confirmamos que el archivo fue creado correctamente.

Archivo CSV creado exitosamente: df_mod_anemia.csv


### ENCUESTA 4: PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON ANEMIA GRAVE

In [ ]:
import requests
import pandas as pd
import json

# URL base de la API de DHS (Nutrición - Anemia infantil moderada)
base_url = "https://api.dhsprogram.com/rest/dhs/data/CN_ANMC_C_SEV"

# Definir los parámetros para la solicitud (si soporta paginación)
params = {
    "perpage": 1000,  # Número de registros por página (se solicita un máximo de 1000 registros por solicitud)
    "page": 1         # Página inicial, comenzamos desde la primera página de resultados
}

# Lista para almacenar todos los datos que se vayan obteniendo
all_data = []

# Hacer varias solicitudes hasta obtener todos los datos (paginación)
while True:
    # Realizar la solicitud a la API utilizando los parámetros definidos
    response = requests.get(base_url, params=params)
    
    # Verificar si la solicitud fue exitosa (código de estado 200)
    if response.status_code == 200:
        # Convertir la respuesta en formato JSON para manejar los datos
        data = response.json()
        
        # Extraer la lista de datos ("Data") de la respuesta JSON y agregarla a all_data
        all_data.extend(data.get("Data", []))
        
        # Verificar si hemos recibido menos registros de los solicitados, lo que indica que hemos llegado al final
        if len(data.get("Data", [])) < params["perpage"]:
            break
        
        # Incrementar el número de página para la siguiente solicitud
        params["page"] += 1
    else:
        # En caso de error, imprimir el código de estado de la respuesta
        print("Error al obtener los datos:", response.status_code)
        break

# Convertir los datos obtenidos (all_data) a un DataFrame de Pandas para facilitar su análisis
df_sev = pd.json_normalize(all_data)

# Guardar el DataFrame en un archivo CSV
output_file = "df_sev_anemia.csv"
df_sev.to_csv(output_file, index=False, encoding="utf-8")

# Imprimir mensaje de éxito indicando que el archivo CSV fue creado correctamente
print(f"Archivo CSV creado exitosamente: {output_file}")


Archivo CSV creado exitosamente: df_sev_anemia.csv


# CUARTA FUENTE: KAGGLE

In [ ]:
pip install kaggle

In [ ]:
import json
import os

# Ruta al archivo 'kaggle.json' que contiene las credenciales de acceso a la API de Kaggle
kaggle_file_path = 'kaggle.json'

# Leer el archivo JSON que contiene las credenciales
with open(kaggle_file_path, 'r') as file:
    kaggle_api = json.load(file)

# Extraer el 'username' y la 'key' de las credenciales de Kaggle
kaggle_username = kaggle_api['username']  # Obtiene el nombre de usuario de Kaggle
kaggle_key = kaggle_api['key']  # Obtiene la clave de la API de Kaggle

# Configurar las variables de entorno con las credenciales para acceder a la API de Kaggle
os.environ['KAGGLE_USERNAME'] = kaggle_username  # Establece la variable de entorno para el nombre de usuario
os.environ['KAGGLE_KEY'] = kaggle_key  # Establece la variable de entorno para la clave de la API

# Imprimir un mensaje de confirmación indicando que las credenciales han sido configuradas correctamente
print("Credenciales de Kaggle configuradas correctamente.")

In [ ]:
import subprocess

# Ejecutar el comando para listar datasets relacionados con "child anemia" en Kaggle y capturar la salida
result = subprocess.run(
    ["kaggle", "datasets", "list", "-s", "child anemia"],  # El comando Kaggle para buscar datasets
    capture_output=True,  # Captura tanto la salida estándar como los errores
    text=True  # Decirle a subprocess que el resultado debe ser tratado como texto (en lugar de bytes)
)

# Imprimir la salida del comando ejecutado
print(result.stdout)  # Muestra el resultado del comando (en este caso, la lista de datasets)



### DATA 1: Factores que afectan el nivel de anemia en los niños (Estudio en Nigeria)


In [ ]:
import subprocess

# Nombre del dataset a descargar desde Kaggle
dataset_name = "adeolaadesina/factors-affecting-children-anemia-level"

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

# Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "children anemia.csv"  # El nombre del archivo CSV descargado

# Leer el archivo CSV en un DataFrame de Pandas
df_nigeria = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso Nigeria)")


### DATA 2: Encuesta Nacional de Familia y Salud (Estudio en ¿India?)

In [ ]:
import subprocess

# Nombre del dataset que quieres descargar desde Kaggle
dataset_name = "ravisinghiitbhu/nfhs5"  # Identificador del dataset

# Ejecutar el comando para descargar y descomprimir el dataset
subprocess.run(["kaggle", "datasets", "download", "-d", dataset_name, "--unzip"])


In [ ]:
import pandas as pd

## Verifica el nombre del archivo descargado y ajústalo aquí
csv_file = "Final.csv"  # Nombre del archivo CSV descargado desde Kaggle

# Leer el archivo CSV en un DataFrame de Pandas
df_india = pd.read_csv(csv_file)

# Confirmar que los datos se han leído correctamente
print("Datos extraídos exitosamente de Kaggle (Caso India)")


# **Módulo Análisis de Datos**

## API: World Bank Prevalencia de anemia infantil (% de anemia infantil entre los 6-59 meses)

## 1. Limpieza de Datos
### 1.1 Primera revisión del archivo para observar sus columnas y contenido de filas

In [ ]:
import pandas as pd

# Cargar el archivo CSV con los datos sobre anemia del Banco Mundial
file_path = 'world_bank_anemia.csv'  # Ruta al archivo CSV
data = pd.read_csv(file_path)

# Mostrar una vista previa de los datos para verificar la carga y la estructura
print("Primeras 5 filas del DataFrame:")
print(data.head())

print("\nInformación general del DataFrame:")
print(data.info())

# Obtener los nombres de las columnas para su posterior análisis
columnas = data.columns.tolist()
print("\nNombres de las columnas:")
print(columnas)


### 1.2 Eliminamos las columnas innecesarias y filas sin valores de anemia, también redondeamos valores de anemia

In [ ]:
# Cargar el archivo CSV
archivo_original = "world_bank_anemia.csv"
df = pd.read_csv(archivo_original)

# Especificar columnas a eliminar (ajusta según tus necesidades)
columnas_a_eliminar = ["unit", "obs_status", "decimal", "indicator.id",
                      "countryiso3code", "country.id", "indicator.value"]

# Limpiar y transformar los datos
df['value'] = df['value'].round(1)  # Redondear a 1 decimal
df_filtrado = df.drop(columns=columnas_a_eliminar)  # Eliminar columnas innecesarias
world_bank_anemia_limpio = df_filtrado.dropna(subset=['value'])  # Eliminar filas con valores faltantes en 'value'

# Verificar los resultados
print(world_bank_anemia_limpio.head())  # Mostrar las primeras filas del DataFrame limpio

## 2. Filtrar
### 2.1 Filtrar CSV: Separación por países

In [ ]:
# Cargar el archivo CSV original
archivo_csv = "world_bank_anemia_limpio.csv"  # Reemplaza con la ruta a tu archivo
df = pd.read_csv(archivo_csv)

# Filtrar las filas desde "Afghanistan" hasta "Zimbabwe" en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
filtro_paises = world_bank_anemia_limpio[ultima_columna].str.strip().isin(["Afghanistan", "Zimbabwe"])  # Filtrar inicio y fin

# Obtener los índices del rango
inicio = world_bank_anemia_limpio[filtro_paises].index.min()  # Índice de "Afghanistan"
fin = world_bank_anemia_limpio[filtro_paises].index.max()  # Índice de "Zimbabwe"

# Seleccionar los datos dentro de este rango
df_paises = world_bank_anemia_limpio.iloc[inicio:fin + 1]  # Incluye ambas filas

### 2.1.1 Mejoramos el nombre de las columnas:

In [ ]:
# Renombrar las columnas para una mejor comprensión
df_paises = df_paises.rename(columns={
    'date': 'year',  # Cambia 'date' a 'year'
    'value': 'prevalencia (%)',  # Cambia 'value' a 'prevalencia (%)'
    'country.value': 'pais'  # Cambia 'country.value' a 'pais'
})

# Guardar el DataFrame modificado como un nuevo archivo CSV
archivo_csv_nuevo = "world_bank_anemia_paises_listo.csv"
df_paises.to_csv(archivo_csv_nuevo, index=False)  # Guardar sin incluir el índice

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}")

### 2.2 Agrupar por continentes

In [ ]:
# Cargar el archivo CSV
file_path = 'world_bank_anemia_paises_listo.csv'
data = pd.read_csv(file_path)

# Diccionario para mapear países a continentes
country_to_continent = {
    "Afghanistan": "Asia",
    "Albania": "Europe",
    "Algeria": "Africa",
    "Andorra": "Europe",
    "Angola": "Africa",
    "Antigua and Barbuda": "North America",
    "Argentina": "South America",
    "Armenia": "Asia",
    "Australia": "Oceania",
    "Austria": "Europe",
    "Azerbaijan": "Asia",
    "Bahamas": "North America",
    "Bahamas, The": "North America",
    "Bahrain": "Asia",
    "Bangladesh": "Asia",
    "Barbados": "North America",
    "Belarus": "Europe",
    "Belgium": "Europe",
    "Belize": "North America",
    "Benin": "Africa",
    "Bhutan": "Asia",
    "Bolivia": "South America",
    "Bosnia and Herzegovina": "Europe",
    "Botswana": "Africa",
    "Brazil": "South America",
    "Brunei Darussalam": "Asia",
    "Bulgaria": "Europe",
    "Burkina Faso": "Africa",
    "Burundi": "Africa",
    "Cabo Verde": "Africa",
    "Cambodia": "Asia",
    "Cameroon": "Africa",
    "Canada": "North America",
    "Central African Republic": "Africa",
    "Chad": "Africa",
    "Chile": "South America",
    "China": "Asia",
    "Colombia": "South America",
    "Comoros": "Africa",
    "Congo, Dem. Rep.": "Africa",
    "Congo, Rep.": "Africa",
    "Costa Rica": "North America",
    "Cote d'Ivoire": "Africa",
    "Croatia": "Europe",
    "Cuba": "North America",
    "Cyprus": "Asia",
    "Czech Republic": "Europe",
    "Czechia": "Europe",
    "Denmark": "Europe",
    "Djibouti": "Africa",
    "Dominica": "North America",
    "Dominican Republic": "North America",
    "Ecuador": "South America",
    "Egypt": "Africa",
    "Egypt, Arab Rep.": "Africa",
    "El Salvador": "North America",
    "Equatorial Guinea": "Africa",
    "Eritrea": "Africa",
    "Estonia": "Europe",
    "Eswatini": "Africa",
    "Ethiopia": "Africa",
    "Fiji": "Oceania",
    "Finland": "Europe",
    "France": "Europe",
    "Gabon": "Africa",
    "Gambia": "Africa",
    "Gambia, The": "Africa",
    "Georgia": "Asia",
    "Germany": "Europe",
    "Ghana": "Africa",
    "Greece": "Europe",
    "Grenada": "North America",
    "Guatemala": "North America",
    "Guinea": "Africa",
    "Guinea-Bissau": "Africa",
    "Guyana": "South America",
    "Haiti": "North America",
    "Honduras": "North America",
    "Hungary": "Europe",
    "Iceland": "Europe",
    "India": "Asia",
    "Indonesia": "Asia",
    "Iran": "Asia",
    "Iran, Islamic Rep.": "Asia",
    "Iraq": "Asia",
    "Ireland": "Europe",
    "Israel": "Asia",
    "Italy": "Europe",
    "Jamaica": "North America",
    "Japan": "Asia",
    "Jordan": "Asia",
    "Kazakhstan": "Asia",
    "Kenya": "Africa",
    "Kiribati": "Oceania",
    "Korea, Dem. People's Rep.": "Asia",
    "Korea, Dem. Rep.": "Asia",
    "Korea, Rep.": "Asia",
    "Kuwait": "Asia",
    "Kyrgyz Republic": "Asia",
    "Kyrgyzstan": "Asia",
    "Lao PDR": "Asia",
    "Latvia": "Europe",
    "Lebanon": "Asia",
    "Lesotho": "Africa",
    "Liberia": "Africa",
    "Libya": "Africa",
    "Liechtenstein": "Europe",
    "Lithuania": "Europe",
    "Luxembourg": "Europe",
    "Madagascar": "Africa",
    "Malawi": "Africa",
    "Malaysia": "Asia",
    "Maldives": "Asia",
    "Mali": "Africa",
    "Malta": "Europe",
    "Marshall Islands": "Oceania",
    "Mauritania": "Africa",
    "Mauritius": "Africa",
    "Mexico": "North America",
    "Micronesia, Fed. Sts.": "Oceania",
    "Moldova": "Europe",
    "Monaco": "Europe",
    "Mongolia": "Asia",
    "Montenegro": "Europe",
    "Morocco": "Africa",
    "Mozambique": "Africa",
    "Myanmar": "Asia",
    "Namibia": "Africa",
    "Nauru": "Oceania",
    "Nepal": "Asia",
    "Netherlands": "Europe",
    "New Zealand": "Oceania",
    "Nicaragua": "North America",
    "Niger": "Africa",
    "Nigeria": "Africa",
    "North Macedonia": "Europe",
    "Norway": "Europe",
    "Oman": "Asia",
    "Pakistan": "Asia",
    "Palau": "Oceania",
    "Panama": "North America",
    "Papua New Guinea": "Oceania",
    "Paraguay": "South America",
    "Peru": "South America",
    "Philippines": "Asia",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Qatar": "Asia",
    "Romania": "Europe",
    "Russian Federation": "Europe",
    "Rwanda": "Africa",
    "Samoa": "Oceania",
    "San Marino": "Europe",
    "Sao Tome and Principe": "Africa",
    "Saudi Arabia": "Asia",
    "Senegal": "Africa",
    "Serbia": "Europe",
    "Seychelles": "Africa",
    "Sierra Leone": "Africa",
    "Singapore": "Asia",
    "Slovak Republic": "Europe",
    "Slovenia": "Europe",
    "Solomon Islands": "Oceania",
    "Somalia": "Africa",
    "South Africa": "Africa",
    "South Sudan": "Africa",
    "Spain": "Europe",
    "Sri Lanka": "Asia",
    "St. Kitts and Nevis": "North America",
    "St. Lucia": "North America",
    "St. Vincent and the Grenadines": "North America",
    "Sudan": "Africa",
    "Suriname": "South America",
    "Sweden": "Europe",
    "Switzerland": "Europe",
    "Syrian Arab Republic": "Asia",
    "Tajikistan": "Asia",
    "Tanzania": "Africa",
    "Thailand": "Asia",
    "Timor-Leste": "Asia",
    "Togo": "Africa",
    "Tonga": "Oceania",
    "Trinidad and Tobago": "North America",
    "Tunisia": "Africa",
    "Turkey": "Asia",
    "Turkmenistan": "Asia",
    "Tuvalu": "Oceania",
    "Uganda": "Africa",
    "Ukraine": "Europe",
    "United Arab Emirates": "Asia",
    "United Kingdom": "Europe",
    "United States": "North America",
    "Uruguay": "South America",
    "Uzbekistan": "Asia",
    "Vanuatu": "Oceania",
    "Venezuela": "South America",
    "Venezuela, RB": "South America",
    "Vietnam": "Asia",
    "Viet Nam": "Asia",
    "West Bank and Gaza": "Asia",
    "Yemen": "Asia",
    "Yemen, Rep.": "Asia",
    "Zambia": "Africa",
    "Zimbabwe": "Africa"
}

#Este diccionario define la asociación entre cada país y su respectivo continente. Los nombres de los países deben coincidir con los nombres en la columna `country.value` del archivo CSV cargado.

# Crear la nueva columna "Continente"
data['Continente'] = data['country.value'].map(country_to_continent) #Se agrega una nueva columna al DataFrame llamada `Continente`. Para cada país en la columna `country.value`, se asigna el continente correspondiente utilizando el diccionario `country_to_continent`.


# Ordenar los países por continente y luego por nombre
sorted_data = data.sort_values(by=['Continente', 'country.value']) #Los datos se ordenan primero por el continente (columna `Continente`) y luego por el nombre del país (columna `country.value`) de manera alfabética.

# Guardar el archivo CSV resultante
output_path = 'world_bank_continentes.csv'
sorted_data.to_csv(output_path, index=False)  #Se guarda el DataFrame resultante en un nuevo archivo CSV llamado `world_bank_continentes.csv`. La opción `index=False` asegura que el índice del DataFrame no se incluya en el archivo.

print(f"Archivo generado: {output_path}") #Se imprime un mensaje en la consola confirmando la generación del archivo CSV.

### 2.3 Filtrar CSV: Separación por ingresos

In [ ]:
# Lista de valores a filtrar en la última columna
valores_deseados = [
    "High income",
    "Low & middle income",
    "Low income",
    "Middle income",
    "Upper middle income"
]

# Filtrar las filas que contienen estos valores en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna 
#Se identifica el nombre de la última columna del DataFrame `world_bank_anemia_limpio` accediendo al último elemento de la lista de nombres de columnas mediante el índice `-1`.

world_bank_anemia_filtrado = world_bank_anemia_limpio[world_bank_anemia_limpio[ultima_columna].str.strip().isin(valores_deseados)]
#En esta línea, se realiza el filtrado del DataFrame:
#- `world_bank_anemia_limpio[ultima_columna]`: Selecciona los valores de la última columna.
#- `.str.strip()`: Elimina espacios en blanco al inicio y al final de cada valor, asegurando coincidencias precisas.
#- `.isin(valores_deseados)`: Verifica si cada valor pertenece a la lista `valores_deseados`.
#- Las filas que cumplen estas condiciones se seleccionan y se asignan a un nuevo DataFrame llamado `world_bank_anemia_filtrado`.

### 2.3.1 Mejoramos el nombre de las columnas y valor de filas:

In [ ]:
# Cambiar los nombres de las columnas
world_bank_anemia_filtrado = world_bank_anemia_filtrado.rename(columns={
    'date': 'year',
    'value': 'prevalencia (%)',
    'country.value': 'nivel de ingresos'
})
#En esta parte, se renombra las columnas del DataFrame `world_bank_anemia_filtrado` de la siguiente manera:
#- `date` pasa a llamarse `year`.
#- `value` pasa a llamarse `prevalencia (%)`.
#- `country.value` pasa a llamarse `nivel de ingresos`.

# Modificar los valores de la columna "nivel de ingresos"
world_bank_anemia_filtrado['nivel de ingresos'] = world_bank_anemia_filtrado['nivel de ingresos'].replace({
    'High income': 'Ingresos altos',
    'Low & middle income': 'Ingresos bajos y medios',
    'Low income': 'Bajos ingresos',
    'Middle income': 'Ingreso medio',
    'Upper middle income': 'Ingreso medio alto',
    'Lower middle income': 'Ingreso medio bajo'
})
#Los valores de la columna `nivel de ingresos` se traducen o modifican según el siguiente mapeo:
#- `High income` se convierte en `Ingresos altos`.
#- `Low & middle income` se convierte en `Ingresos bajos y medios`.
#- `Low income` se convierte en `Bajos ingresos`.
#- `Middle income` se convierte en `Ingreso medio`.
#- `Upper middle income` se convierte en `Ingreso medio alto`.
#- `Lower middle income` se convierte en `Ingreso medio bajo`.

#La función `replace` permite realizar estas transformaciones en toda la columna.


# Guardar el nuevo archivo CSV con los nombres de columnas cambiados
archivo_csv_nuevo = "world_bank_anemia_ingresos_listo.csv"
world_bank_anemia_filtrado.to_csv(archivo_csv_nuevo, index=False)
#El DataFrame modificado se guarda en un nuevo archivo CSV llamado `world_bank_anemia_ingresos_listo.csv`. La opción `index=False` asegura que no se incluya el índice del DataFrame en el archivo.

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}") #Se imprime un mensaje en la consola para confirmar que el archivo se ha guardado correctamente.

### 2.4 Filtrar CSV: A nivel mundial

In [ ]:
# Lista de valores a filtrar en la última columna
valores_deseados = [
    "World"
]

# Filtrar las filas que contienen estos valores en la última columna
ultima_columna = world_bank_anemia_limpio.columns[-1]  # Nombre de la última columna
df_filtrado = world_bank_anemia_limpio[world_bank_anemia_limpio[ultima_columna].str.strip().isin(valores_deseados)]

# Guardar en un nuevo archivo CSV
nuevo_csv = "world_bank_anemia_mundial.csv"
df_filtrado.to_csv(nuevo_csv, index=False)

print(f"Datos filtrados guardados en: {nuevo_csv}")

### 2.4.1 Mejoramos el nombre de las columnas y valor de filas:

In [ ]:
# Cargar el archivo CSV
archivo_csv = "world_bank_anemia_mundial.csv"  # Asegúrate de que el archivo esté en la misma carpeta o proporciona la ruta completa
df = pd.read_csv(archivo_csv)
#Se carga el archivo CSV denominado `world_bank_anemia_mundial.csv` en un DataFrame llamado `df` utilizando la función `read_csv` de `pandas`.
#- Si el archivo no está en el directorio de trabajo actual, se debe proporcionar la ruta completa.

# Cambiar los nombres de las columnas
df = df.rename(columns={
    'date': 'year',
    'value': 'prevalencia (%)',
    'country.value': 'nivel geográfico'
})
#Las columnas se renombran de la siguiente manera:
#- `date` pasa a llamarse `year`.
#- `value` pasa a llamarse `prevalencia (%)`.
#- `country.value` pasa a llamarse `nivel geográfico`.
#Esto se realiza mediante el método `rename`, proporcionando un diccionario que mapea los nombres antiguos a los nuevos.

# Modificar los valores de la columna "nivel geográfico"
df['nivel geográfico'] = df['nivel geográfico'].replace({
    'World': 'Mundial'
})
#En la columna `nivel geográfico`, el valor `World` se reemplaza por `Mundial` para hacer el texto más comprensible.

# Guardar el nuevo archivo CSV con los nombres de columnas cambiados
archivo_csv_nuevo = "world_bank_anemia_mundial_listo.csv"
df.to_csv(archivo_csv_nuevo, index=False)
#El DataFrame modificado se guarda en un nuevo archivo CSV denominado `world_bank_anemia_mundial_listo.csv`.
#- El parámetro `index=False` asegura que el índice del DataFrame no se incluya como una columna adicional en el archivo generado.

print(f"Archivo con las columnas renombradas guardado como {archivo_csv_nuevo}")
#Se imprime un mensaje en la consola confirmando que el archivo se ha guardado correctamente.

# API: Demographic Health Survey (PORCENTAJE DE NIÑOS MENORES A 5 AÑOS CLASIFICADOS CON CUALQUIER/LEVE/MODERADO/SEVERO NIVEL DE ANEMIA)

In [ ]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
# Cargar archivo para anemia general
file_path = 'df_any_anemia.csv'
any = pd.read_csv(file_path)

#Cargar archivo para anemia leve
file_path = 'df_mld_anemia.csv'
mild = pd.read_csv(file_path)

#Cargar archivo para anemia moderada
file_path = 'df_mld_anemia.csv'
mod = pd.read_csv(file_path)

#Cargar archivo para anemia severa
file_path = 'df_mld_anemia.csv'
sev = pd.read_csv(file_path)

## 1. Limpieza de Datos

### 1.1 Eliminación de columnas innecesarias

In [ ]:
# Lista de columnas a eliminar
columns_to_drop = ['DataId', 'SurveyId', 'Indicator', 'IsPreferred', 'SDRID', 'Precision', 'RegionId', 'SurveyType',
'IndicatorId', 'CharacteristicOrder', 'CharacteristicLabel',  'ByVariableLabel', 'CIHigh', 'IsTotal', 'ByVariableId',
                   'IndicatorOrder', 'DHS_CountryCode',  'CILow', 'LevelRank', 'CharacteristicId', 'CharacteristicCategory'
                 , 'IndicatorType',
                   'DenominatorUnweighted','DenominatorWeighted', "SurveyYearLabel", "Value"
]

# Eliminar las columnas no deseadas
df_cleaned0 = any.drop(columns=columns_to_drop)
df_cleaned1 = mild.drop(columns=columns_to_drop)
df_cleaned2 = mod.drop(columns=columns_to_drop)
df_cleaned3 = sev.drop(columns=columns_to_drop)

## 1.2 Mejorar nombres de columnas

In [ ]:
# Crear un nuevo dataframe con las columnas especificadas
df_combined = pd.DataFrame({
    'Year': any['SurveyYear'],  # SurveyYear de 'any'
    'Pais': any['CountryName'],  # CountryName de 'any'

    'Valor Cualquier': any['Value'],  # Value de 'any' renombrado
    '# Encuestas (any, sin ponderar)': any['DenominatorUnweighted'],  # DenominatorUnweighted de 'any'
    '# Encuestas (any, ponderadas)': any['DenominatorWeighted'],  # DenominatorWeighted de 'any'

    'Valor Leve': mild['Value'],  # Value de 'mild' renombrado
    '# Encuestas (mild, sin ponderar)': mild['DenominatorUnweighted'],  # DenominatorUnweighted de 'mild'
    '# Encuestas (mild, ponderadas)': mild['DenominatorWeighted'],  # DenominatorWeighted de 'mild'

    'Valor Moderado': mod['Value'],  # Value de 'mod' renombrado
    '# Encuestas (mod, sin ponderar)': mod['DenominatorUnweighted'],  # DenominatorUnweighted de 'mod'
    '# Encuestas (mod, ponderadas)': mod['DenominatorWeighted'],  # DenominatorWeighted de 'mod'

    'Valor Severo': sev['Value'],  # Value de 'sev' renombrado
    '# Encuestas (sev, sin ponderar)': sev['DenominatorUnweighted'],  # DenominatorUnweighted de 'sev'
    '# Encuestas (sev, ponderadas)': sev['DenominatorWeighted']  # DenominatorWeighted de 'sev'
})



# Guardar el dataframe combinado como un nuevo archivo CSV
output_file_combined = 'dhs_anemia_final.csv'
df_combined.to_csv(output_file_combined, index=False)

# Imprimir la ruta del archivo guardado
print(f"Archivo guardado en: {output_file_combined}")

# API KAGGLE (FACTORES QUE PODRIAN ESTAR INFLUENCIANDO EL NIVEL DE ANEMIA EN NIÑOS DE 0-59 MESES) - Caso: Nigeria

## Descripción del caso

En este estudio, se recopilaron datos transversales de las Encuestas demográficas y de salud de Nigeria (NDHS) de 2018 para responder a preguntas de investigación sobre el efecto de la edad de las madres y otros factores socioeconómicos en el nivel de anemia de los niños de 0 a 59 meses en Nigeria. Las DHS son encuestas transversales de hogares representativas a nivel nacional que generalmente se realizan cada 5 años. Los datos de esta encuesta consideraron los 36 estados de Nigeria, así como el Territorio de la Capital Federal (FCT). La población objetivo de este estudio son los niños de 0 a 59 meses y las madres de 15 a 49 años. En esta encuesta, el ingreso del hogar se midió utilizando el índice de riqueza, la edad actual en grupos de 5 años se produce agrupando la edad actual en años completados, tipo de lugar de residencia donde el encuestado fue entrevistado como urbano o rural, la categorización se creó en función de si el número de punto de muestra o conglomerado se define como urbano o rural, el nivel más alto de educación alcanzado es una variable estandarizada que proporciona el nivel de educación en las siguientes categorías: Sin educación, Educación primaria, secundaria y superior, el número total de nacimientos en los últimos cinco años se define como todos los nacimientos en los meses 0 a 59 anteriores al mes de la entrevista, donde el mes 0 es el mes de la entrevista, la edad del encuestado en el primer nacimiento se calcula utilizando el CMC de la fecha de nacimiento del encuestado.

Después de la depuración de los datos, se utilizó el método Chi cuadrado para probar las hipótesis sobre la posible relación que existe entre ciertos factores socioeconómicos y los niveles de anemia en niños de 0 a 59 meses. El nivel de anemia fue la variable predictora y las variables explicativas son la edad de la madre, el nivel de educación, el índice de riqueza, el nacimiento en los últimos cinco años, el uso de mosquiteros, etc.

# Diccionario de datos

```Python
Type of place of residence
0: Rural
1: Urban


Highest educational level
0: Higher
1: No education
2: Primary
3: Secondary


Wealth index combined
0: Middle
1: Poorer
2: Poorest
3: Richer
4: Richest


Anemia level
0: Mild
1: Moderate
2: Not anemic
3: Severe


Have mosquito bed net for sleeping (from household questionnaire)
0: No
1: Yes


Smokes cigarettes
0: No
1: Yes


Current marital status
0: Divorced
1: Living with partner
2: Married
3: Never in union
4: No longer living together/separated
5: Widowed


Currently residing with husband/partner
0: Living with her
1: Staying elsewhere


When child put to breast
0: 102.0
1: 103.0
2: 104.0
... (continuando con valores similares)
38: Days: 1
39: Hours: 1
40: Immediately


Had fever in last two weeks
0: Don't know
1: No
2: Yes


Taking iron pills, sprinkles or syrup
0: Don't know
1: No
2: Yes
```





## 1. Implementacion de funciones para limpiar, ordenar y transformar los datos.

In [ ]:
import pandas as pd

# Cargar el archivo proporcionado por el usuario
file_path = 'children anemia.csv'
data = pd.read_csv(file_path)

# Mostrar una vista previa de los datos para analizar la estructura
data.head(), data.info()

In [ ]:
# Funciones para limpiar, ordenar y transformar los datos
def limpiar_datos(dataframe):
    """
    Limpia los datos eliminando columnas duplicadas, renombrando columnas y gestionando valores faltantes.
    """
    # Eliminar columnas duplicadas o irrelevantes
    columnas_a_eliminar = ['Hemoglobin level adjusted for altitude and smoking (g/dl - 1 decimal)',
                           'Anemia level.1']
    dataframe = dataframe.drop(columns=columnas_a_eliminar, errors='ignore')

    # Renombrar columnas para mayor claridad
    dataframe = dataframe.rename(columns={
        'Age in 5-year groups': 'Age_Group',
        'Type of place of residence': 'Residence_Type',
        'Highest educational level': 'Education_Level',
        'Wealth index combined': 'Wealth_Index',
        'Births in last five years': 'Births_Last_5_Years',
        'Age of respondent at 1st birth': 'Age_First_Birth',
        'Anemia level': 'Anemia_Level',
        'Have mosquito bed net for sleeping (from household questionnaire)': 'Mosquito_Net',
        'Smokes cigarettes': 'Smokes',
        'Current marital status': 'Marital_Status',
        'Currently residing with husband/partner': 'Residing_With_Partner',
        'When child put to breast': 'Breastfeeding_Timing',
        'Had fever in last two weeks': 'Fever_Last_2_Weeks',
        'Hemoglobin level adjusted for altitude (g/dl - 1 decimal)': 'Hemoglobin_Level',
        'Taking iron pills, sprinkles or syrup': 'Iron_Supplements'
    })

    # Manejo de valores faltantes
    dataframe['Anemia_Level'] = dataframe['Anemia_Level'].fillna('Unknown')  # Llenar valores faltantes de anemia con "Unknown"
    dataframe = dataframe.dropna(subset=['Hemoglobin_Level', 'Education_Level'])  # Eliminar filas con valores críticos faltantes

    return dataframe


def transformar_datos(dataframe):
    """
    Transforma los datos categóricos a variables numéricas y estandariza las columnas.
    """
    # Convertir categorías a valores numéricos
    categoricas_a_codificar = ['Residence_Type', 'Education_Level', 'Wealth_Index', 'Anemia_Level',
                               'Mosquito_Net', 'Smokes', 'Marital_Status', 'Residing_With_Partner',
                               'Breastfeeding_Timing', 'Fever_Last_2_Weeks', 'Iron_Supplements']

    for columna in categoricas_a_codificar:
        dataframe[columna] = dataframe[columna].astype('category').cat.codes

    return dataframe


# Cargar el archivo CSV
file_path = 'children anemia.csv'  # Reemplaza con la ruta del archivo en tu sistema
data = pd.read_csv(file_path)

# Aplicar funciones al dataset
datos_limpios = limpiar_datos(data)
datos_transformados = transformar_datos(datos_limpios)

# Mostrar las primeras filas del DataFrame transformado
print(datos_transformados.head())

# Guardar los datos transformados en un archivo CSV
datos_transformados.to_csv('datos_limpios_transformados.csv', index=False)
print("Datos guardados como 'datos_limpios_transformados.csv'.")

## 2. Calcular métricas como media, mediana, moda, y otras estadísticas relevantes

In [ ]:
def calcular_metricas_avanzadas(dataframe):
    """
    Calcula métricas descriptivas avanzadas como percentiles, asimetría, curtosis y más.
    """
    # Seleccionar columnas numéricas
    columnas_numericas = dataframe.select_dtypes(include=['int64', 'float64'])

    # Crear un diccionario para almacenar las métricas
    estadisticas = {}

    for columna in columnas_numericas.columns:
        estadisticas[columna] = {
            'Media': columnas_numericas[columna].mean(),
            'Mediana': columnas_numericas[columna].median(),
            'Moda': columnas_numericas[columna].mode().iloc[0] if not columnas_numericas[columna].mode().empty else None,
            'Desviación Estándar': columnas_numericas[columna].std(),
            'Mínimo': columnas_numericas[columna].min(),
            'Máximo': columnas_numericas[columna].max(),
            'Rango': columnas_numericas[columna].max() - columnas_numericas[columna].min(),
            'Percentil 25': columnas_numericas[columna].quantile(0.25),
            'Percentil 75': columnas_numericas[columna].quantile(0.75),
            'Asimetría': columnas_numericas[columna].skew(),
            'Curtosis': columnas_numericas[columna].kurt(),
            'Rango Intercuartílico (IQR)': columnas_numericas[columna].quantile(0.75) - columnas_numericas[columna].quantile(0.25),
            'Coeficiente de Variación (CV)': columnas_numericas[columna].std() / columnas_numericas[columna].mean(),
        }

    # Convertir a un DataFrame para mejor visualización
    estadisticas_df = pd.DataFrame(estadisticas).transpose()
    return estadisticas_df


# Calcular métricas avanzadas
metricas_avanzadas = calcular_metricas_avanzadas(datos_transformados)

# Mostrar las métricas avanzadas
print(metricas_avanzadas)


## 3. Estructuras de datos (listas, pilas, colas) que faciliten el análisis

### 3.1 Lista

Las listas son ideales para almacenar datos tabulares o registros específicos que se necesitan procesar en secuencia

In [ ]:
# Extraer datos relevantes y almacenarlos en una lista
lista_anemia = datos_transformados[['Age_Group', 'Anemia_Level', 'Hemoglobin_Level']].values.tolist()

# Ejemplo de acceso a los datos
print("Ejemplo de registros en lista:")
print(lista_anemia[:5])  # Imprime los primeros 5 registros


### 3.2 Pila (Stack)

Las pilas siguen el principio LIFO (Last In, First Out) y son útiles si los datos se necesitan procesar en orden inverso.

In [ ]:
class Pila:
    # Constructor de la clase Pila
    def __init__(self):
        self.stack = [] # Inicializamos una lista vacía que será nuestra pila

    # Método para agregar un elemento a la pila
    def push(self, item):
        self.stack.append(item) # Agrega el elemento al final de la lista (pila)

    # Método para eliminar y retornar el último elemento de la pila
    def pop(self):
        # Si la pila no está vacía, eliminamos el último elemento. Si está vacía, devolvemos None
        return self.stack.pop() if not self.is_empty() else None

    # Método para ver el último elemento de la pila sin eliminarlo
    def peek(self):
        # Si la pila no está vacía, retornamos el último elemento. Si está vacía, devolvemos None
        return self.stack[-1] if not self.is_empty() else None

    # Método para verificar si la pila está vacía
    def is_empty(self):
        return len(self.stack) == 0  # La pila está vacía si su longitud es 0

# Crear una pila con los datos relevantes
pila_anemia = Pila()
# Iterar sobre las filas del DataFrame 'datos_transformados'
for _, row in datos_transformados.iterrows():
    # Crear un diccionario con el grupo de edad y nivel de anemia y agregarlo a la pila
    pila_anemia.push({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la pila
print("Dato extraído de la pila:", pila_anemia.pop())


### 3.3 Cola (Queue)

Las colas siguen el principio FIFO (First In, First Out) y son útiles si los datos se deben procesar en el mismo orden en que se almacenaron.

In [ ]:
from collections import deque

# Crear una cola con los datos relevantes
cola_anemia = deque()

# Iterar sobre las filas del DataFrame 'datos_transformados'
for _, row in datos_transformados.iterrows():
    # Crear un diccionario con el grupo de edad y el nivel de anemia y agregarlo a la cola
    cola_anemia.append({'Age_Group': row['Age_Group'], 'Anemia_Level': row['Anemia_Level']})

# Ejemplo de uso de la cola
print("Dato extraído de la cola:", cola_anemia.popleft())


## API: Global Health Observatory - Prevalencia de anemia infantil (% de anemia infantil entre los 6-59 meses)

## 1. Analisis de datos inicial

In [ ]:
import requests
import pandas as pd

#Se importan las librerías `requests` para realizar solicitudes HTTP y `pandas` para manipular y analizar los datos en formato tabular.

# URL de la API para la prevalencia de anemia (sustituir con el código correcto)
indicator_code = "NUTRITION_ANAEMIA_CHILDREN_PREV"  # Cambia a la API para prevalencia de anemia
url = f"https://ghoapi.azureedge.net/api/{indicator_code}"

#- Se define el código del indicador de la API (`NUTRITION_ANAEMIA_CHILDREN_PREV` en este caso).
#- Se construye la URL para realizar la solicitud a la API utilizando el código del indicador.

# Realizar la solicitud
response = requests.get(url)
#Se utiliza el método `get` de la librería `requests` para realizar una solicitud HTTP a la URL de la API.


# Verificar que la solicitud fue exitosa
if response.status_code == 200:
    data = response.json()  # Extraer los datos en formato JSON
#- Se verifica si la solicitud fue exitosa comprobando que el código de estado HTTP es `200`.
#- Si la respuesta es exitosa, los datos se extraen en formato JSON utilizando el método `json` de la respuesta.

    # Convertir los datos a un DataFrame
    df = pd.json_normalize(data['value'])
    #- Los datos extraídos del JSON (ubicados en la clave `value`) se convierten en un DataFrame de `pandas` usando `json_normalize`.

    # Mostrar las primeras filas
    print(df.head(48))
    #Se imprime una vista previa de las primeras 48 filas del DataFrame para verificar la estructura de los datos.


else:
    print(f"Error al acceder a la API: {response.status_code}")
    #- Si la solicitud no es exitosa, se imprime un mensaje de error junto con el código de estado HTTP para facilitar la depuración.

## **2. Limpieza de datos**


In [ ]:
#Este fragmento de código verifica si hay valores nulos en el DataFrame y muestra la cantidad de valores nulos por columna.
print(df.isnull().sum())

##### Explicación detallada
#1. **`df.isnull()`**:
#   - Este método de pandas genera un nuevo DataFrame del mismo tamaño que `df`, donde cada celda contiene `True` si el valor correspondiente en el DataFrame original es nulo (`NaN`) y `False` si no lo es.

#2. **`.sum()`**:
#   - Este método se aplica a lo largo de las columnas del DataFrame generado por `isnull()`. Cuenta el número de valores `True` en cada columna, es decir, la cantidad de valores nulos por columna.

#3. **`print()`**:
#   - El resultado del conteo de valores nulos por columna se imprime en la salida estándar para que el usuario pueda revisarlo.

### **2.1 Eliminación de filas y columnas sin datos**

*   Elemento de la lista
*   Elemento de la lista

In [ ]:
#Este código elimina valores nulos en un DataFrame utilizando el método `dropna()` de pandas. Se aborda tanto la eliminación de filas como de columnas que contienen valores nulos.

# Eliminar filas con valores nulos
df_cleaned = df.dropna()

# Eliminar columnas con valores nulos
df_cleaned = df.dropna(axis=1)

#### Explicación detallada
#1. **`df.dropna()`**:
#   - Este método elimina las filas del DataFrame que contienen al menos un valor nulo (`NaN`).
#   - Devuelve un nuevo DataFrame (`df_cleaned`), sin modificar el DataFrame original (`df`).

#2. **`df.dropna(axis=1)`**:
#   - Este método elimina las columnas del DataFrame que contienen al menos un valor nulo.
#   - El argumento `axis=1` indica que la operación se realiza sobre las columnas (en lugar de las filas, que es el valor predeterminado).

#3. **Sobrescritura del DataFrame limpio**:
#   - En este código, `df_cleaned` se sobrescribe al eliminar filas y luego columnas con valores nulos. Si necesitas conservar ambos resultados (sin filas nulas y sin columnas nulas), asigna cada operación a variables diferentes.

In [ ]:
#Este código imprime los nombres de las columnas del DataFrame `df_cleaned` después de realizar una limpieza, como la eliminación de filas y columnas con valores nulos.

print("Columnas después de la limpieza:")
print(df_cleaned.columns)

#### Explicación detallada
#1. **`print("Columnas después de la limpieza:")`**:
#   - Este comando imprime un mensaje descriptivo que indica al usuario que se mostrarán las columnas actuales del DataFrame tras la limpieza.

#2. **`print(df_cleaned.columns)`**:
#   - Este comando imprime los nombres de las columnas del DataFrame `df_cleaned`.
#   - El atributo `columns` de un DataFrame devuelve un objeto `Index` que contiene los nombres de todas las columnas.

### **2.2 Imputación de Datos**

In [ ]:
# Seleccionar columnas específicas
# Asegúrate de que los nombres de las columnas coincidan exactamente con los del DataFrame
columnas_deseadas = ['SpatialDimType', 'SpatialDim', 'Dim1Type', 'TimeDim','NumericValue', 'Low', 'High']
# Elimina espacios extra en ' Dim1Type' y 'TimeDim '

# Verificar si las columnas existen antes de seleccionarlas
columnas_existentes = df_cleaned.columns
columnas_deseadas = [col for col in columnas_deseadas if col in columnas_existentes]

df_seleccion = df_cleaned[columnas_deseadas]

print(df_seleccion.head(30))


### Explicación detallada
#1. **Definir columnas deseadas**:
#   - Se crea una lista `columnas_deseadas` con los nombres de las columnas que queremos seleccionar.
#   - Asegúrate de que los nombres de las columnas coincidan exactamente con los nombres en el DataFrame original (incluyendo espacios en blanco, si los hubiera).

#2. **Verificar columnas existentes**:
#   - El código obtiene todas las columnas actuales del DataFrame `df_cleaned` usando `df_cleaned.columns`.
#- Filtra la lista `columnas_deseadas` para conservar solo aquellas columnas que existen en el DataFrame.


#3. **Seleccionar columnas**:
#   - Usa la lista filtrada `columnas_deseadas` para crear un nuevo DataFrame `df_seleccion` que contiene solo las columnas deseadas.

#4. **Imprimir el resultado**:
#  - Muestra las primeras 30 filas del DataFrame `df_seleccion` utilizando el método `head(30)`.

## 3. Organización de datos

In [ ]:
import pandas as pd

#Este código realiza una limpieza y estandarización en la columna `SpatialDim` de un DataFrame.
#Se asegura de que la columna contenga cadenas de texto, filtra valores no numéricos para encontrar la moda (el valor más frecuente) y
#reemplaza los valores numéricos con la moda. Finalmente, muestra los primeros 30 registros del DataFrame para verificar el resultado.


# Asegurando que la columna tiene valores correctos
# Aquí supongo que df es tu DataFrame
df['SpatialDim'] = df['SpatialDim'].astype(str)

# Filtrar los valores no numéricos y encontrar la moda
moda_no_numerica = df[~df['SpatialDim'].str.isnumeric()]['SpatialDim'].mode()[0]

# Reemplazar los valores numéricos por la moda
df['SpatialDim'] = df['SpatialDim'].apply(lambda x: moda_no_numerica if x.isnumeric() else x)

# Mostrar los primeros registros para verificar
print(df_seleccion.head(30))

### Explicación detallada
#1. **Convertir valores a cadenas**:
#   - Se asegura que todos los valores en la columna `SpatialDim` sean cadenas utilizando `astype(str)`. Esto es importante para garantizar que las operaciones posteriores funcionen correctamente.

#2. **Filtrar valores no numéricos**:
#   - Se utiliza el método `~df['SpatialDim'].str.isnumeric()` para seleccionar los valores que no son numéricos.
#   - Se calcula la moda (el valor más frecuente) entre los valores no numéricos usando `.mode()[0]`.

#3. **Reemplazar valores numéricos**:
#   - La función `apply` evalúa cada elemento de la columna `SpatialDim`.
#   - Si el valor es numérico (`x.isnumeric()`), lo reemplaza con la moda no numérica. De lo contrario, conserva el valor original.

#4. **Mostrar resultado**:
#   - Se imprimen las primeras 30 filas del DataFrame modificado para verificar los cambios realizados.

### **3.1 Obtener años únicos por país en un DataFrame ordenado**

In [ ]:
# Agrupar los años únicos por cada valor de SpatialDim
years_by_spatialdim = df[df['SpatialDimType'] == 'COUNTRY'].groupby('SpatialDim')['TimeDim'].unique()
#Este fragmento de código years_by_spatialdim = df[df['SpatialDimType'] == 'COUNTRY'] filtra el DataFrame df para seleccionar solo las filas en las que el valor de la columna SpatialDimType sea 'COUNTRY'.
#Después de filtrar, se agrupan los datos por los valores de la columna SpatialDim.
#Para cada grupo (valor único en SpatialDim), se extraen los valores únicos de la columna TimeDim (los años), utilizando .unique()
# Convertir a un DataFrame para una mejor presentación

result = years_by_spatialdim.apply(sorted).reset_index(name='Years')
#Los años (que son valores únicos) se ordenan para que el resultado esté en un formato ascendente
#El resultado de la agrupación se convierte en un DataFrame, restableciendo el índice y renombrando la columna con los años como Years.

# Mostrar el resultado
print(result)
#Finalmente, se imprime el DataFrame resultante que contiene una columna SpatialDim y una columna Years, que contiene los años únicos asociados a cada valor de SpatialDim.

#### **3.2 COMPARAR AÑOS CON MAYOR Y MENOR PROMEDIO USANDO LISTAS**

In [ ]:
# Lista para almacenar promedios anuales
year_averages = []
#Se crea una lista vacía llamada year_averages,
#que se usará para almacenar las tuplas que contienen el año y su respectivo promedio de la columna NumericValue.

# Calcular promedio por año
for year in df['TimeDim'].unique():
    values = list(df[df['TimeDim'] == year]['NumericValue'])
    avg = sum(values) / len(values)
    year_averages.append((year, avg))

#Se itera sobre cada año único en la columna TimeDim (df['TimeDim'].unique()), lo que asegura que se procesa cada año presente en los datos.
#Para cada año, se filtran los valores correspondientes de la columna NumericValue (df[df['TimeDim'] == year]['NumericValue']) y se convierten en una lista.
#Se calcula el promedio de esos valores utilizando sum(values) / len(values).
#Se agrega una tupla (año, promedio) a la lista year_averages

# Ordenar por promedio
year_averages.sort(key=lambda x: x[1])

#Se ordena la lista year_averages en función del valor del promedio (el segundo elemento de cada tupla) utilizando sort() con una función lambda que extrae el segundo elemento (x[1]).

# Mostrar resultados
print(f"Año con menor promedio: {year_averages[0][0]} - Promedio: {year_averages[0][1]:.2f}")
print(f"Año con mayor promedio: {year_averages[-1][0]} - Promedio: {year_averages[-1][1]:.2f}")

#Se imprime el año con el menor promedio (year_averages[0]) y el año con el mayor promedio (year_averages[-1]).
#El formato :.2f asegura que los promedios se muestren con dos decimales.

#### **4.1 PREVALENCIA DE ANEMIA POR CADA AÑO EN LOS DIFERENTES PAISES**

In [ ]:
# Filtrar registros donde SpatialDimType sea "COUNTRY"
df_filtered = df[df['SpatialDimType'] == 'COUNTRY']
#Este fragmento filtra el DataFrame df para seleccionar solo las filas en las que el valor de la columna SpatialDimType es igual a 'COUNTRY'.
#El resultado es almacenado en un nuevo DataFrame df_filtered.

# Calcular el promedio de prevalencia de anemia por país y año
average_anemia = df_filtered.groupby(['SpatialDim', 'TimeDim'])['NumericValue'].mean().reset_index()
#El DataFrame filtrado df_filtered se agrupa por las columnas SpatialDim (representando el país) y TimeDim (representando el año).
#Para cada combinación de país y año, se calcula el promedio de la columna NumericValue, que en este caso se asume que representa la prevalencia de anemia.
#El resultado de la agrupación y el cálculo del promedio se convierte en un nuevo DataFrame con reset_index() para restaurar el índice y hacerlo más accesible

# Renombrar las columnas para mayor claridad
average_anemia.columns = ['Country', 'Year', 'AveragePrevalence']

#Las columnas del DataFrame average_anemia se renombran a Country, Year y AveragePrevalence para mejorar la legibilidad y comprensión de los datos.

# Mostrar los primeros registros del resultado
print(average_anemia.head(30))

#Finalmente, se imprimen las primeras 30 filas del DataFrame average_anemia utilizando .head(30).
#Esto proporciona una vista preliminar de los resultados del cálculo de los promedios



### 4.1.1 Descargar el archivo en un csv

In [ ]:
from google.colab import files
import pandas as pd

#Se importa la librería files de Google Colab, que permite interactuar con archivos en el entorno de Colab.
#También se importa pandas como pd, que es utilizado para manejar y manipular el DataFrame average_anemia

# ... (Your existing code to generate average_anemia DataFrame) ...

# Save the DataFrame to a CSV file
average_anemia.to_csv('PREVALENCIA_DE_ANEMIA_POR_CADA_AÑO_EN_LOS_DIFERENTES_PAISES.csv', index=False)
#Se guarda el DataFrame average_anemia en un archivo CSV. El nombre del archivo será 'PREVALENCIA_DE_ANEMIA_POR_CADA_AÑO_EN_LOS_DIFERENTES_PAISES.csv'.
#El argumento index=False asegura que el índice del DataFrame no se incluya en el archivo CSV, para evitar la creación de una columna extra con los índices de las filas.

# Download the CSV file
files.download('PREVALENCIA_DE_ANEMIA_POR_CADA_AÑO_EN_LOS_DIFERENTES_PAISES.csv')

#El archivo CSV generado se descarga automáticamente al entorno local del usuario mediante la función files.download().
#El nombre del archivo descargado será el mismo que el utilizado en el paso anterior

#### **4.2 PROMEDIO DE PREVALENCIA POR PAÍS USANDO LISTAS**

In [ ]:
 # Lista para almacenar los promedios
country_averages = []
#Se crea una lista vacía llamada country_averages que almacenará las tuplas con el nombre del país y su correspondiente promedio de prevalencia de anemia.

# Agrupar por país y calcular el promedio manualmente
for country in df['SpatialDim'].unique():
    if df[df['SpatialDim'] == country]['SpatialDimType'].iloc[0] == 'COUNTRY':
        prevalences = list(df[df['SpatialDim'] == country]['NumericValue'])
        avg_prevalence = sum(prevalences) / len(prevalences)
        country_averages.append((country, avg_prevalence))

#Se itera sobre cada país único en la columna SpatialDim utilizando df['SpatialDim'].unique(). Esto asegura que cada país sea procesado una vez.
#Dentro de la iteración, se verifica si el valor de SpatialDimType para ese país es 'COUNTRY' en la primera fila de los registros del país (df[df['SpatialDim'] == country]['SpatialDimType'].iloc[0]). Esto garantiza que solo se procesen los países.
#Para cada país, se obtienen los valores de prevalencia de anemia de la columna NumericValue correspondientes a ese país, convirtiéndolos en una lista con list(df[df['SpatialDim'] == country]['NumericValue']).
#Se calcula el promedio de las prevalencias utilizando la fórmula sum(prevalences) / len(prevalences).
#Se agrega una tupla (país, promedio de prevalencia) a la lista country_averages.

# Mostrar los resultados
for country, avg in country_averages:
    print(f"{country}: {avg:.2f}")
#Después de calcular los promedios para todos los países, se itera sobre la lista country_averages, y para cada tupla (country, avg),
# se imprime el país y su promedio de prevalencia de anemia, con el promedio mostrado con dos decimales usando :.2f

### 4.2.1 Descargar el archivo en un csv

In [ ]:
# Crear un DataFrame a partir de los resultados
results_df = pd.DataFrame(country_averages, columns=['Country', 'AveragePrevalence'])
#Se crea un nuevo DataFrame results_df a partir de la lista country_averages, que contiene tuplas con el país y su promedio de prevalencia de anemia.
#Cada tupla tiene dos elementos: el nombre del país y el promedio de prevalencia.
#Se especifican los nombres de las columnas como ['Country', 'AveragePrevalence'] para que el DataFrame tenga una estructura clara y fácil de entender

# Guardar el DataFrame como archivo CSV
results_df.to_csv('PROMEDIO_DE_PREVALENCIA_POR_PAÍS_USANDO_LISTAS_gho.csv', index=False)
#El DataFrame results_df se guarda en un archivo CSV con el nombre 'PROMEDIO_DE_PREVALENCIA_POR_PAÍS_USANDO_LISTAS_gho.csv'.
#El parámetro index=False asegura que el índice del DataFrame no se incluya como una columna adicional en el archivo CSV, resultando en un archivo más limpio y fácil de leer.

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('PROMEDIO_DE_PREVALENCIA_POR_PAÍS_USANDO_LISTAS_gho.csv')

#Se importa la librería files de Google Colab, que permite interactuar con archivos en el entorno de Colab.
#La función files.download() se utiliza para descargar el archivo CSV generado a la máquina local del usuario.
#Esto facilita la transferencia del archivo desde Colab a un entorno local para su uso posterior.

#### **4.3 RANGO DE PREVALENCIA POR AÑO USANDO LISTAS**

In [ ]:
import pandas as pd

# Crear una pila para almacenar rangos por año
ranges_stack = []
#Se crea una lista vacía llamada ranges_stack,
#que se usará para almacenar tuplas que contienen el año, el valor mínimo, el valor máximo y el rango (diferencia entre el máximo y el mínimo) de los valores de prevalencia para cada año.

# Agrupar por año
# Iterar sobre los años únicos en la columna 'TimeDim' para calcular los valores por año
for year in df['TimeDim'].unique():
    # Obtener los valores de prevalencia de anemia para el año actual
    values = list(df[df['TimeDim'] == year]['NumericValue'])
    # Calcular el valor máximo y mínimo para el año
    max_val = max(values)
    min_val = min(values)
    # Calcular el rango como la diferencia entre el valor máximo y el mínimo
    year_range = max_val - min_val
    # Almacenar los resultados (año, mínimo, máximo y rango) en la lista
    ranges_stack.append((year, min_val, max_val, year_range))

# Crear un DataFrame con las columnas correspondientes
ranges_df = pd.DataFrame(ranges_stack, columns=['Año', 'Min', 'Max', 'Rango'])

# Mostrar el DataFrame
print(ranges_df)

###### **4.3.1 DESCARGAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
ranges_df.to_csv('RANGO_DE_PREVALENCIA_POR_AÑO_USANDO_LISTAS_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('RANGO_DE_PREVALENCIA_POR_AÑO_USANDO_LISTAS_gho.csv')

#### **4.4 CALCULAR LA MEDIANA DE PREVALENCIA USANDO COLAS**

In [ ]:
import pandas as pd
from collections import deque

# Calcular la mediana por año
medians_queue = deque()
#Se importa deque de la librería collections.
#deque es una estructura de datos eficiente para agregar y eliminar elementos de ambos extremos de una lista.
#En este caso, se usa para almacenar los resultados de las medianas por año.

# Agrupar por año
# Iterar sobre los años únicos en la columna 'TimeDim' para calcular la mediana por año
for year in df['TimeDim'].unique():
    # Obtener y ordenar los valores de prevalencia de anemia para el año actual
    values = sorted(list(df[df['TimeDim'] == year]['NumericValue']))
    # Calcular el número de valores
    n = len(values)
    # Calcular la mediana según si el número de valores es par o impar
    if n % 2 == 0:  # Si hay un número par de valores
        median = (values[n // 2 - 1] + values[n // 2]) / 2
    else:  # Si hay un número impar de valores
        median = values[n // 2]
    # Almacenar el año y la mediana en la cola
    medians_queue.append((year, median))

# Convertir los resultados a un DataFrame
medians_df = pd.DataFrame(medians_queue, columns=['Año', 'Mediana'])

# Mostrar el DataFrame
print(medians_df)

##### **4.4.1 GUARDAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
medians_df.to_csv('LA_MEDIANA_DE_PREVALENCIA_USANDO_COLAS_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('LA_MEDIANA_DE_PREVALENCIA_USANDO_COLAS_gho.csv')

#### **4.5 COMPARAR PAÍSES CON MÁS ESTABILIDAD**

In [ ]:
import pandas as pd

# Lista vacía para almacenar el rango de prevalencia por país
country_ranges = []

# Calcular rango por país
for country in df['SpatialDim'].unique():
    # Verificar que el tipo de dimensión espacial sea 'COUNTRY'
    if df[df['SpatialDim'] == country]['SpatialDimType'].iloc[0] == 'COUNTRY':
        # Obtener los valores de prevalencia de anemia para el país
        values = list(df[df['SpatialDim'] == country]['NumericValue'])
        # Calcular el rango como la diferencia entre el valor máximo y el mínimo
        country_range = max(values) - min(values)
        # Almacenar el país y su rango en la lista
        country_ranges.append((country, country_range))

# Ordenar por rango
country_ranges.sort(key=lambda x: x[1])

# Convertir a un DataFrame
ranges_df = pd.DataFrame(country_ranges, columns=['País', 'Rango'])

# Mostrar el DataFrame
print(ranges_df)  # Mostrar los 60 países más estables

#### **4.5.1 DESCARGAR EN FORMATO CSV**

In [ ]:
# Guardar el DataFrame como archivo CSV
ranges_df.to_csv('COMPARAR_PAÍSES_CON_MÁS_ESTABILIDAD_gho.csv', index=False)

# Descargar el archivo a tu ordenador
from google.colab import files
files.download('COMPARAR_PAÍSES_CON_MÁS_ESTABILIDAD_gho.csv')

## 5. Visualización de datos

### 5.1 Paquetes requeridos

In [ ]:
# - pandas: libreria para la manipulación y análisis de datos.
import pandas as pd
# - matplotlib: es una librería utilizada para crear visualizaciones estáticas, animadas e interactivas.
# - pyplot: proporciona una interfaz de scripting similar a MATLAB.
import matplotlib.pyplot as plt
# - numpy: librería que soporta arrays y matrices multidimensionales.
import numpy as np
# - ipywidgets: para crear widgets interactivos en Jupyter Notebooks para mejorar la interactividad de las celdas.
import ipywidgets as widgets
# - plotly.express: para hacer graficos interactivos, no funcionaba en mapamundi.
import plotly.express as px 
# - interact: facilita la creación de widgets interactivos que se actualizan dinámicamente en función de las entradas del usuario.
from ipywidgets import interact
# - display: se utiliza para mostrar objetos en la salida del Jupyter Notebook.
# - clear_output: se utiliza para limpiar la salida.
from IPython.display import display, clear_output
# - Wedge: se utiliza para crear figuras en forma de cuña en los gráficos, como en gráficos circulares o de pastel.
from matplotlib.patches import Wedge
# - table: se utiliza para mostrar DataFrames de pandas como tablas en gráficos matplotlib.
from pandas.plotting import table
# - make_subplots: para poder poner subgraficos categorizados.
from plotly.subplots import make_subplots
# - go: contiene herramientas para crear gráficos complejos y detallados con plotly.
import plotly.graph_objects as go
# - random: para generar números aleatorios en Python.
import random
# - seaborn: proporciona una interfaz de alto nivel para dibujar gráficos estadísticos atractivos y complejos.
import seaborn as sns

#Para el mapamundi
# se instala la libreria folium para crear mapas interactivos y visualizaciones geoespaciales.
#!pip install folium
#import folium

# se instala la libreria geopandas para facilitar el trabajo con datos geoespaciales.
#!pip install geopandas
# se instala la libreria geopy para facilitar la geocodificación (convertir direcciones en coordenadas geográficas) y la realización de cálculos geográficos.
#!pip install geopy

#No se podía poner el filtro dinámico, gpt comentó instalar --> #jupyter nbextension enable --py widgetsnbextension
#generaba un error porque la versión de Jupyter no lo soportaba.
#Por lo que se actualizo con este código --> pip install --upgrade jupyter jupyterlab ipywidgets

### 5.2 Anemia infantil a través del tiempo (2000-2019)

#### 5.2.1 Anemia infantil a nivel mundial

In [ ]:
# fuente: Banco Mundial

# cargar datos: utiliza pandas para leer el archivo CSV llamado world_bank_anemia_mundial_listo.csv y almacenarlo en un DataFrame llamado data_historico
data_historico = pd.read_csv(r"world_bank_anemia_mundial_listo.csv")

# crea un gráfico de líneas utilizando plotly.express.
fig = px.line(
    data_historico, # DataFrame que contiene los datos.
    x="year", # define el eje X como la columna "year".
    y="prevalencia (%)", # define el eje Y como la columna "prevalencia (%)".
    markers=True, # agrega marcadores a los puntos de datos.
    title="Prevalencia de anemia (en %), 2000-2019", # agrega titulo al gráfico.
    labels={"year": "Año", "prevalencia (%)": "Prevalencia (%)"} # etiquetas personalizadas para los ejes.
)

# personalizar las trazas del gráfico.
fig.update_traces(
    hovertemplate="Prevalencia: %{y:.1f}%", # etiquetas personalizadas para los ejes.
    line=dict(color="blue"), # cambia el color de la línea a azul.
    marker=dict(size=8) # ajusta el tamaño de los marcadores a 8.
)
# personalizar el diseño del gráfico.
fig.update_layout(
    yaxis=dict(title="Prevalencia (%)", range=[10, 50]), # título del eje Y y rango de valores.
    xaxis=dict(
        tickangle=-90,  # rotar las etiquetas de los años.
        tickmode='array',  # mostrar todos los años.
        tickvals=data_historico["year"],  # asegura que todos los años se muestren.
        title=""  # eliminar el título del eje X.
    ),
    title=dict(font=dict(size=18)), # tamaño de fuente del título del gráfico.
    template="simple_white" # estilo del gráfico.
)

# mostrar el gráfico interactivo en la salida del notebook.
fig.show()

#### 5.2.2 Prevalencia de anemia infantil según nivel de ingreso

In [ ]:
#Fuente: Banco Mundial

# cargar datos: utiliza pandas para leer el archivo CSV llamado world_bank_anemia_ingresos_listo.csv y almacenarlo en un DataFrame llamado data_nivelingresos.
data_nivelingresos = pd.read_csv(r"world_bank_anemia_ingresos_listo.csv")

# convertir los valores de la columna 'year' a numéricos usando la función pd.to_numeric, reemplazando cualquier valor no convertible con NaN.
data_nivelingresos['year'] = pd.to_numeric(data_nivelingresos['year'], errors='coerce')
# elimina las filas con datos faltantes en las columnas 'year' y 'prevalencia (%)' usando la función dropna.
data_nivelingresos = data_nivelingresos.dropna(subset=['year', 'prevalencia (%)'])
# convertir los valores de la columna 'year' a enteros usando la función astype(int)
data_nivelingresos['year'] = data_nivelingresos['year'].astype(int)

# crea un gráfico de líneas interactivo utilizando plotly.express.
fig = px.line(data_nivelingresos, # DataFrame que contiene los datos.
              x='year', # define el eje X como la columna 'year'.
              y='prevalencia (%)', # define el eje Y como la columna 'prevalencia (%)'.
              color='nivel de ingresos', # colorea las líneas según el nivel de ingresos.
              markers=True, # agrega marcadores a los puntos de datos.
              title='Prevalencia histórica de anemia por nivel de ingresos', # título del gráfico.
              labels={'prevalencia (%)': 'Prevalencia (%)', 'year': 'Año'}) # etiquetas personalizadas para los ejes.

# eliminar la leyenda del gráfico.
fig.update_layout(showlegend=False)

# añadir el nombre del nivel de ingresos cerca del último punto disponible y alineado verticalmente.
y_offset = 0.1  # ajuste de distancia entre los nombres de los niveles.
for i, level in enumerate(data_nivelingresos['nivel de ingresos'].unique()):
    level_data = data_nivelingresos[data_nivelingresos['nivel de ingresos'] == level]
    last_row = level_data[level_data['year'] == level_data['year'].max()]
    if not last_row.empty:
        last_year = last_row['year'].values[0]
        last_value = last_row['prevalencia (%)'].values[0]
        
        # añadir la anotación en la posición deseada.
        fig.add_annotation(
            x=last_year + 0.4,  # posición horizontal cerca del último punto.
            y=last_value + (y_offset * i),  # posición vertical con ajuste para separar los nombres.
            text=level,  # texto con el nombre del nivel de ingresos.
            showarrow=False,  # sin flecha.
            font=dict(size=10),  # tamaño de la fuente.
            xanchor='left',  # alineación del texto a la izquierda.
            align='left',  # alineación del texto a la izquierda.
        )

# mejorar el diseño del gráfico.
# mode='lines+markers': asegura que las líneas y los marcadores se muestren.
# hovertemplate='Prevalencia: %{y}': solo muestra la prevalencia en el tooltip.
fig.update_traces(mode='lines+markers', hovertemplate='Prevalencia: %{y}')
fig.update_layout(
    xaxis_title='Año', # título del eje X.
    yaxis_title='Prevalencia (%)', # título del eje Y.
    xaxis=dict(
        title=None,  # quitar el título del eje X.
        tickmode='array', # define el modo de los ticks.
        tickvals=sorted(
            data_nivelingresos['year'].unique()), # asegura que todos los años se muestren.
        showline=True, # muestra la línea del eje.
        linecolor='black', # color de la línea del eje.
        ticks='outside',  # muestra las marcas de graduación hacia afuera.
        tickwidth=1,  # define el grosor de las marcas.
    ),
    yaxis=dict(
            showline=True,  # muestra la línea del eje Y.
            linewidth=1,  # define el grosor de la línea.
            linecolor='black'  # define el color de la línea.
    ),
    xaxis_tickangle=-90,  # rota las etiquetas del eje X 90 grados.
    plot_bgcolor='white', # define el color de fondo del gráfico.
    font=dict(size=12), # define el tamaño de la fuente.
    title_x=0.5,  # centrar el título
)

# mostrar el gráfico interactivo en la salida del notebook.
fig.show()

#### 5.3.3 Prevalencia de anemia infantil por país (Multiselección) 

In [ ]:
# Fuente: Banco Mundial

# Cargar datos: Utiliza pandas para leer el archivo CSV llamado world_bank_anemia_ingresos_listo.csv y almacenarlo en un DataFrame llamado data_historico_pais_est.
data_historico_pais_est = pd.read_csv(r"world_bank_anemia_paises_listo.csv")
data_historico_pais_est['year'] = pd.to_numeric(data_historico_pais_est['year'], errors='coerce')
data_historico_pais_est['year'] = data_historico_pais_est['year'].astype(int)

# Obtener una lista de países únicos y los ordena alfabéticamente.
countries = sorted(data_historico_pais_est['pais'].unique())

# Asignar un color único a cada país.
def assign_colors(countries): # asigna un color aleatorio (en formato RGBA) a cada país y devuelve un diccionario con los colores.
    colors = {}
    for country in countries:
        colors[country] = f'rgba({random.randint(0,255)},{random.randint(0,255)},{random.randint(0,255)}, 0.8)'
    return colors

colors = assign_colors(countries)

# Función para completar los años faltantes y hacer líneas continuas.
def completar_anios(data, country):
    # Filtrar datos del país.
    country_data = data[data['pais'] == country].copy()

    # Generar el rango completo de años.
    all_years = pd.DataFrame({'year': range(country_data['year'].min(), country_data['year'].max() + 1)})

    # Unir con los datos originales y llenar los valores faltantes mediante interpolación.
    completed_data = pd.merge(all_years, country_data, on='year', how='left')
    completed_data['prevalencia (%)'] = completed_data['prevalencia (%)'].interpolate()

    # Añadir el nombre del país.
    completed_data['pais'] = country
    return completed_data

# Función para graficar prevalencias históricas de anemia basadas en los países seleccionados.
def plot_selected_countries_plotly(countries_selected):
    if not countries_selected:
        with output_area:
            clear_output(wait=True)
            print("Por favor selecciona al menos un país.")
        return

    # Crear una figura.
    fig = go.Figure()

    for country in countries_selected:
        # Completar los años faltantes.
        country_data = completar_anios(data_historico_pais_est, country)

        # Dividir los datos en tres segmentos.
        before_2020 = country_data[country_data['year'] < 2020]
        between_2020_2030 = country_data[(country_data['year'] >= 2020) & (country_data['year'] <= 2030)]
        after_2030 = country_data[country_data['year'] > 2030]

        # Obtener el color para el país.
        country_color = colors[country]

        # Añadir el segmento antes de 2020 (línea continua).
        fig.add_trace(
            go.Scatter(
                x=before_2020['year'],
                y=before_2020['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'.
                line=dict(color=country_color)  # Usamos el color del país.
            )
        )

        # Añadir el segmento entre 2020 y 2030 (línea punteada).
        fig.add_trace(
            go.Scatter(
                x=between_2020_2030['year'],
                y=between_2020_2030['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'.
                line=dict(dash='dot', color=country_color)  # Usamos el mismo color del país para la línea punteada.
            )
        )

        # Añadir el segmento después de 2030 (línea continua).
        fig.add_trace(
            go.Scatter(
                x=after_2030['year'],
                y=after_2030['prevalencia (%)'],
                mode='lines+markers',
                name=country,
                hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",  # Personalizar el tooltip sin el símbolo '%'.
                line=dict(color=country_color)  # Usamos el color del país.
            )
        )

        # Colocar el nombre del país ligeramente desplazado a la derecha de 2030.
        year_2030_data = country_data[country_data['year'] == 2019]
        if not year_2030_data.empty:
            # Obtenemos el valor de prevalencia para 2030.
            prev_2030 = year_2030_data['prevalencia (%)'].values[0]
            fig.add_annotation(
                x=2019 + 1.2,  # Desplazamos un poco a la derecha de 2030.
                y=prev_2030,
                text=country,
                showarrow=False,
                font=dict(size=10, color='black'),
                xanchor='left',  # Alineación del texto a la izquierda.
                align='left'  # Alineación del texto a la izquierda.
            )

    # Ajustar el diseño del gráfico.
    fig.update_layout(
        title={
            'text': 'Prevalencia histórica de anemia',
            'x': 0.5,  # Centrar el título.
            'xanchor': 'center',  # Asegurar que el anclaje sea en el centro.
        },
        xaxis=dict(
            title=None,  # Quitar el título del eje X.
            tickangle=-90, # Rotar las etiquetas del eje X 90 grados.
            showline=True, # Mostrar la línea del eje X.
            linecolor='black', # Definir el color de la línea del eje X.
            ticks='outside',  # Mostrar marcas de graduación principales hacia el exterior.
            tickwidth=1,  # Grosor de las marcas de graduación.
            tickvals=list(
                range(
                    data_historico_pais_est['year'].min(), 
                    data_historico_pais_est['year'].max() + 1
                )
            )  # Asegurar que todos los años estén en el eje X.
        ),
        yaxis=dict(
            showline=True,  # Mostrar la línea del eje Y.
            linewidth=1,  # Definir el grosor de la línea.
            linecolor='black'  # Definir el color de la línea.
        ),
        showlegend=False,  # Quitar la leyenda.
        yaxis_title='Prevalencia (%)', # Definir el título del eje Y.
        legend_title='Países', # Definir el titulo de la leyenda
        template='plotly_white', # Aplica la plantilla de diseño 'plotly_white'.
        width=850  # Establece el ancho del gráfico a 850 píxeles.
    )

    # Mostrar el gráfico en el área de salida.
    with output_area:
        # Limpia la salida existente en el área de salida antes de mostrar el nuevo gráfico.
        clear_output(wait=True)
        # Muestra el gráfico interactivo.
        fig.show()

# Función para manejar la actualización dinámica del gráfico.
def update_plot(change=None):
    # Obtener los países seleccionados.
    selected_countries = [cb.description for cb in checkboxes if cb.value]

    # Actualizar el gráfico.
    plot_selected_countries_plotly(selected_countries)

# Crear checkboxes para cada país.
# Usar ipywidgets para crear una lista de checkboxes, donde cada checkbox corresponde a un país y está inicialmente desmarcada
checkboxes = [widgets.Checkbox(value=False, description=country) for country in countries]

# Agregar un evento a cada checkbox.
for cb in checkboxes:
    cb.observe(update_plot, names='value')

# Crear un contenedor para los checkboxes con barra de desplazamiento.
checkbox_box = widgets.VBox(checkboxes, layout=widgets.Layout(overflow='auto', height='300px', width='200px'))

# Crear un área de salida para el gráfico.
output_area = widgets.Output()

# Mostrar los checkboxes y el área de salida.
display(widgets.HBox([checkbox_box, output_area]))

#### 5.3.4 Clasificación de la importancia de la anemia infantil en la salud pública, por país

In [ ]:
# Fuente: Banco Mundial

# Cargar datos: usar pandas para leer el archivo CSV llamado dhs_anemia_final.csv y almacenarlo en un data frame llamado data_ind_anemia
data_ind_anemia = pd.read_csv(r"dhs_anemia_final.csv")
# Usar la funcion drop para eliminar ciertas columnas del DataFrame.
data_ind_anemia.drop(data_ind_anemia.columns[[3, 4, 5, 6, 7, 8, 9, 10]], axis=1, inplace=True)
# Renombrar las columnas del DataFrame para mayor claridad.
data_ind_anemia.rename(
    columns={
    'Valor Cualquier': 'Valor\nReal',
    'Valor Severo' : 'Valor\nsevero',
    '# Encuestas (sev, sin ponderar)': '# Encuestas\n(sin ponderar)',
    '# Encuestas (sev, ponderadas)': '# Encuestas\n(ponderadas)'
    }, 
    inplace=True) 

# Función para crear un velocímetro estilizado.
def plot_stylized_gauge(ax, value, country, min_val=0, max_val=100):
    levels = ["Baja", "Moderada", "Alta"] # Niveles del velocímetro.
    colors = ["#32CD32", "#FFD700", "#FF4D4D"]  # Colores verde, amarillo y rojo.
    thresholds = [0, 20, 40, 100] # Umbrales de los niveles.

    # Definir el ángulo de inicio y fin del semicírculo.
    start_angle = 180  # Inicio del semicírculo en 180° (sentido antihorario).
    end_angle = 0 # Fin del semicírculo.

    for i, (level, color) in enumerate(zip(levels, colors)):

        # Calcular los ángulos para cada nivel en función de los umbrales.
        start = start_angle - (start_angle - end_angle) * (thresholds[i + 1] - min_val) / (max_val - min_val)
        end = start_angle - (start_angle - end_angle) * (thresholds[i] - min_val) / (max_val - min_val)
        
        # Dibujar sección usando Wedge (un sector circular).
        wedge = Wedge(center=(0, 0), r=1, theta1=start, theta2=end, facecolor=color, edgecolor="white")
        ax.add_patch(wedge)

        # Posicionar texto indicando los niveles.
        mid_angle = (start + end) / 2
        x_text = np.cos(np.radians(mid_angle)) * 0.7
        y_text = np.sin(np.radians(mid_angle)) * 0.7
        ax.text(x_text, y_text, level, ha="center", va="center", fontsize=14, color="white")

    # Dibujar la aguja indicando el valor actual (en sentido antihorario).
    value_angle = start_angle - (start_angle - end_angle) * (value - min_val) / (max_val - min_val)
    x_needle = np.cos(np.radians(value_angle)) * 0.9
    y_needle = np.sin(np.radians(value_angle)) * 0.9
    ax.plot([0, x_needle], [0, y_needle], color="black", linewidth=2)

    # Agregar el valor actual y el nombre del país en el centro del velocímetro.
    ax.text(0, -0.2, f"{value}", ha="center", va="center", fontsize=18, weight="bold")
    ax.text(0, -0.35, country, ha="center", va="center", fontsize=12, style="italic")

    # Ajustar los límites y ocultar ejes.
    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-0.0025, 1.2)  # Mostrar solo el semicírculo superior.
    ax.set_aspect('equal')  # Relación de aspecto 1:1 para un semicírculo perfecto.
    ax.axis("off")

# Función para graficar la tabla de prevalencia.
def plot_paises(paises):
    # Filtrar los datos para el país seleccionado.
    data_paises = data_ind_anemia[data_ind_anemia['Pais'] == paises]
    
    # Obtener el valor "severo" del año más reciente.
    latest_year = data_paises['Year'].max()
    severe_value = data_paises[data_paises['Year'] == latest_year]['Valor\nReal'].values[0] 

    # Crear figura con dos subgráficos (uno para la tabla y otro para el velocímetro).
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))  # Tamaño de la figura.

    # Mostrar la tabla en el primer subgráfico.
    ax1.axis('off')  # Apagar los ejes para la tabla.
    
    # Crear la tabla usando ax.table.
    table_data = data_paises.values
    table_columns = data_paises.columns
    table = ax1.table(cellText=table_data, colLabels=table_columns, loc='center', cellLoc='center')

    # Personalizar el estilo de la tabla.
    table.auto_set_font_size(False)
    table.set_fontsize(12)

    # Ajustar las celdas (bordes, colores, tamaño).
    for (i, j), cell in table.get_celld().items():
        cell.set_edgecolor('black')
        cell.set_linewidth(1.2)
        if i == 0:  # Encabezados.
            cell.set_height(0.25)
            cell.set_facecolor('#4CAF50')
            cell.set_text_props(ha='center', va='center', color='white', weight='bold', fontsize=10)
        else:  # Filas de datos.
            cell.set_facecolor('#f9f9f9')
            cell.set_text_props(color='black', fontsize=10)
        if j == 0:  # Primera columna.
            cell.set_facecolor('#d3d3d3')
            cell.set_text_props(color='black', weight='bold')
            
    # Ajustar el ancho de las columnas basado en el texto.
    col_widths = [max(len(str(cell)) for cell in data_paises[col].tolist() + [col]) for col in data_paises.columns]
    total_width = sum(col_widths)
    for i, width in enumerate(col_widths):
        table.auto_set_column_width(i)  # Ajustar automáticamente.
        table.get_celld()[(0, i)].set_width(width / total_width * 2)  # Normalizar al tamaño del gráfico

    # Mostrar el velocímetro en el segundo subgráfico.
    value = data_paises['Valor\nReal'].values[0]  # Suponiendo que 'Valor Cualquier' es el valor que usas.
    plot_stylized_gauge(ax2, severe_value, paises)

    plt.tight_layout()
    plt.show()

# Crear el selector interactivo
# Genera una interfaz interactiva que permite seleccionar países para mostrar la tabla y el velocímetro.
lista_paises = data_ind_anemia['Pais'].unique()
interact(plot_paises, paises=lista_paises)

### 5.3. **SEGUNDA PARTE = MAPAMUNDI INTERACTIVO (niveles máximos y mínimos de anemia infantil por país y años respectivos)**


In [ ]:
# - pandas: para manipular datos tabulares.
import pandas as pd
# - folium: para crear mapas interactivos.
import folium
# - Nominatim de geopy: para obtener coordenadas geográficas.
from geopy.geocoders import Nominatim
# - tqdm: para mostrar barras de progreso.
from tqdm import tqdm
# - json: para manejar datos en formato JSON.
import json
# - os: para interactuar con el sistema de archivos.
import os

# Cargar datos desde un archivo CSV llamado 'world_bank_continentes.csv' y almacenarlo en un DataFrame llamado df.
df = pd.read_csv(r"world_bank_continentes.csv")
print(df.head()) # Muestra las primeras filas del DataFrame para verificar la estructura de los datos.

# Configurar el geolocalizador usando la API de Nominatim.
geolocator = Nominatim(user_agent="geoapi")

# Definir una función para obtener coordenadas de un país y almacenarlas en un archivo de caché para optimizar futuras consultas.
def obtener_coordenadas(pais, cache_file="coordenadas_cache.json"):
    # Comprueba si ya existe un archivo de caché.
    if os.path.exists(cache_file):
        with open(cache_file, 'r') as f:
            cache = json.load(f) # Carga el contenido del archivo JSON en un diccionario.
    else:
        cache = {} # Crea un diccionario vacío si el archivo no existe.

    # Si el país ya está en el caché, retorna sus coordenadas.
    if pais in cache:
        return cache[pais]

    try:
        location = geolocator.geocode(pais) # Obtiene la ubicación del país usando Nominatim.
        if location:
            coordenadas = (location.latitude, location.longitude) # Obtiene latitud y longitud.
            cache[pais] = coordenadas  # Almacena las coordenadas en el caché.
            # Guarda el caché en un archivo JSON para futuras búsquedas.
            with open(cache_file, 'w') as f:
                json.dump(cache, f)
            return coordenadas
        else:
            return None, None # Si no se encuentra el país, retorna valores nulos.
    except Exception as e:
        print(f"Error obteniendo coordenadas para {pais}: {e}") # Maneja errores de geolocalización.
        return None, None

# Habilitar el uso de barras de progreso con pandas.
tqdm.pandas()  

# Aplicar la función de obtención de coordenadas a cada fila en la columna 'country.value'.
coordenadas = df["country.value"].progress_apply(obtener_coordenadas)

# Desempaquetar las coordenadas en columnas separadas de latitud y longitud.
df["latitude"], df["longitude"] = zip(*coordenadas)

# Mostrar las columnas relevantes para verificar si las coordenadas se agregaron correctamente.
print(df[['country.value', 'latitude', 'longitude']].head())

# Filtrar filas sin coordenadas (si es necesario).
df = df.dropna(subset=["latitude", "longitude"])

# Agrupar datos por país y obtener el máximo y mínimo de prevalencia.
df_grouped = df.groupby('country.value').agg(
    max_prevalencia=('value', 'max'), # Máximo valor de prevalencia.
    min_prevalencia=('value', 'min') # Mínimo valor de prevalencia.
).reset_index()

# Obtener los años correspondientes para los valores máximos y mínimos.
df_max_min_years = df.loc[df.groupby('country.value')['value'].idxmax()][['country.value', 'date']].rename(columns={'date': 'max_year'})
df_min_years = df.loc[df.groupby('country.value')['value'].idxmin()][['country.value', 'date']].rename(columns={'date': 'min_year'})

# Combina los datos agrupados con los años y las coordenadas.
df_map = pd.merge(df_grouped, df_max_min_years, on='country.value')
df_map = pd.merge(df_map, df_min_years, on='country.value')
df_map = pd.merge(df_map, df[['country.value', 'latitude', 'longitude']].drop_duplicates(), on='country.value')

# Verificar si se unieron correctamente las coordenadas y los valores.
print(df_map[['country.value', 'latitude', 'longitude', 'max_prevalencia', 'max_year', 'min_prevalencia', 'min_year']].head())

# Crear un mapa base centrado en coordenadas globales con un nivel de zoom inicial.
world_map = folium.Map(location=[0, 0], zoom_start=2)

# Agregar marcadores agrupados con MarkerCluster.
from folium.plugins import MarkerCluster
marker_cluster = MarkerCluster().add_to(world_map)

# Iterar sobre cada fila del DataFrame combinado y agrega marcadores circulares al mapa.
for _, row in df_map.iterrows():
    tooltip_text = (
        f"País: {row['country.value']}<br>"
        f"Máximo valor: {row['max_prevalencia']} (Año: {row['max_year']})<br>"
        f"Mínimo valor: {row['min_prevalencia']} (Año: {row['min_year']})"
    )

    # Agregar un marcador circular al mapa con información emergente (tooltip).
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']], # Coordenadas del marcador.
        radius=10,  # Radio del círculo.
        color='blue', # Color del borde.
        fill=True, # Habilita el relleno.
        fill_color='cyan', # Color del relleno.
        fill_opacity=0.7, # Opacidad del relleno.
        tooltip=tooltip_text,  # Texto emergente al pasar el cursor.
    ).add_to(marker_cluster)

# Guardar el mapa como un archivo HTML.
world_map.save("mapa_prevalencia_max_min.html")

# Muestra el mapa interactivo en el entorno de ejecución.
world_map

### 5.4. PREDICCIONES HIPOTÉTICAS DE DE LAS PREVALENCIAS DE LA ANEMIA INFANTIL 2020 - 2030 (sin considerar factor covid)

In [ ]:
import pandas as pd # Para cargar, procesar y analizar datos en estructuras como DataFrame.
import matplotlib.pyplot as plt # Para crear visualizaciones como gráficos de líneas, barras, histogramas, etc.
import numpy as np # Para trabajar con arreglos multidimensionales y realizar cálculos matemáticos avanzados.
import ipywidgets as widgets # Para crear elementos interactivos (como deslizadores, menús desplegables, y botones).
import plotly.express as px #Para hacer graficos interactivos, no funcionaba en mapamundi.
from ipywidgets import interact # Para enlazar elementos interactivos (como deslizadores) a funciones Python para que se actualicen dinámicamente
from IPython.display import display, clear_output # - display: Muestra elementos dinámicos o resultados en una celda.
                                                  # - clear_output: Limpia el contenido de salida de una celda para actualizarla con nuevos resultados.
from matplotlib.patches import Wedge # Para crear formas circulares, como secciones de pastel, en gráficos.
from pandas.plotting import table # Para agregar tablas de datos a gráficos de matplotlib.
import plotly.graph_objects as go # Para crear gráficos interactivos avanzados y personalizables.
import random # Para generar números aleatorios y realizar operaciones relacionadas con la aleatoriedad.
from scipy.stats import linregress # realiza análisis de regresión lineal simple, devolviendo parámetros como pendiente, intercepto, valor p, y coeficiente de correlación.

#### 5.4.1. Proyección de anemia infantil a nivel mundial

In [ ]:
# Cargar datos desde un archivo CSV llamado 'world_bank_anemia_mundial_listo.csv' y almacenarlo en un DataFrame llamado data_historico_est
data_historico_est = pd.read_csv(r"world_bank_anemia_mundial_listo.csv")

# Ordenamos los datos por año de forma ascendente (aseguramos que estén en orden cronológico)
data_historico_est = data_historico_est.sort_values(by='year', ascending=True)

# Calcular el factor de crecimiento promedio (promedio de las variaciones porcentuales año tras año)
factor_crecimiento = (data_historico_est['prevalencia (%)'].pct_change().mean() + 1)  # Para que sea un factor de multiplicación

# Lista para almacenar los datos con las estimaciones proyectadas
datos_con_estimaciones = []

# Agregar los datos originales al conjunto de datos de estimaciones
for _, row in data_historico_est.iterrows():
    datos_con_estimaciones.append({
        'year': row['year'],
        'nivel geográfico': row['nivel geográfico'],  # Usar nivel_geografico
        'prevalencia (%)': row['prevalencia (%)']
    })

# Proyectar valores desde 2020 hasta 2030 usando el factor de crecimiento
ultima_prevalencia = data_historico_est['prevalencia (%)'].iloc[-1]  # Obtiene el último valor conocido (2019)

# El último valor de 'nivel_geografico' será el mismo en las proyecciones
nivel_geografico = data_historico_est['nivel geográfico'].iloc[0] # Obtiene el nivel geográfico del primer registro

for year in range(2020, 2031): # Itera desde 2020 hasta 2030
    ultima_prevalencia *= factor_crecimiento  # Aplicar el factor de crecimiento
    datos_con_estimaciones.append({
        'year': year,
        'nivel geográfico': 'Mundial',  # Mantener el mismo nivel_geografico
        'prevalencia (%)': ultima_prevalencia
    })

# Convertir los datos con estimaciones a un DataFrame
data_historico_est = pd.DataFrame(datos_con_estimaciones)

# Reordenar las columnas para que aparezcan como 'year', 'prevalencia (%)' y 'nivel_geografico'
data_historico_est = data_historico_est[['year', 'prevalencia (%)', 'nivel geográfico']]

# Mostrar el DataFrame con las estimaciones
#print(data_historico_est) #Corroborar el resultado

import plotly.graph_objects as go

# Crear el gráfico de línea interactivo para los datos históricos y de proyección
fig = go.Figure()

# Agregar datos históricos al gráfico
fig.add_trace(
    go.Scatter(
        x=data_historico_est[data_historico_est['year'] < 2020]['year'],
        y=data_historico_est[data_historico_est['year'] < 2020]['prevalencia (%)'],
        mode='lines+markers',
        line=dict(color='blue'),
        marker=dict(size=8),
        name=" "
    )
)

# Agregar datos proyectados (2020-2030) al gráfico
fig.add_trace(
    go.Scatter(
        x=data_historico_est[data_historico_est['year'] >= 2020]['year'],
        y=data_historico_est[data_historico_est['year'] >= 2020]['prevalencia (%)'],
        mode='lines+markers',
        line=dict(dash='dot', color='blue'),  # Línea punteada para proyecciones
        marker=dict(size=8),
        name=" "
    )
)

# Configurar las etiquetas y diseño del gráfico
fig.update_traces(
    hovertemplate="Prevalencia: %{y:.1f}%",  # Mostrar solo la prevalencia en el tooltip
    line=dict(color="blue"),
    marker=dict(size=8)
)

# Personalizar diseño del gráfico
fig.update_layout(
    title="Prevalencia de anemia (en %), 2000-2030",  # Título global del gráfico
    xaxis=dict(
        tickangle=-90,  # Rotar las etiquetas de los años
        tickmode='array',  # Mostrar todos los años
        tickvals=data_historico_est["year"],  # Asegura que todos los años se muestren
        title=None
    ),
    yaxis=dict(
        title="Prevalencia (%)",
        range=[10, 50] # Rango fijo para el eje Y
    ),
    template="simple_white",
    showlegend=False  # Ocultar la leyenda
)

# Mostrar el gráfico
fig.show()

#### 5.4.2. Proyección de anemia infantil según el nivel de ingreso

In [ ]:
# Cargar datos desde un archivo CSV llamado 'world_bank_anemia_ingresos_listo.csv'
data = pd.read_csv(r"world_bank_anemia_ingresos_listo.csv")

# Limpiar nombres de columnas (por si tienen espacios adicionales)
data.columns = data.columns.str.strip()

# Lista para almacenar los datos originales y las estimaciones
datos_con_estimaciones = []

# Obtener la lista de niveles de ingresos únicos
niveles_ingresos_unicos = data['nivel de ingresos'].unique()

for nivel in niveles_ingresos_unicos:
    # Filtrar los datos para el nivel de ingresos actual
    datos_nivel = data[data['nivel de ingresos'] == nivel].sort_values(by='year')
    
    # Calcular las variaciones anuales porcentuales
    datos_nivel['variacion'] = datos_nivel['prevalencia (%)'].pct_change()
    
    # Calcular el promedio de la variación porcentual (ignorando valores nulos)
    factor_crecimiento = datos_nivel['variacion'].mean() + 1  # Agregar 1 para obtener el factor multiplicativo
    
    # Agregar los datos originales del nivel de ingresos al conjunto de datos
    for _, row in datos_nivel.iterrows():
        datos_con_estimaciones.append({
            'year': row['year'],
            'nivel de ingresos': row['nivel de ingresos'],
            'prevalencia (%)': row['prevalencia (%)']
        })
    
    # Proyectar valores desde 2020 hasta 2030 usando el factor de crecimiento
    ultima_prevalencia = datos_nivel['prevalencia (%)'].iloc[-1]  # Último valor conocido (2019)
    for year in range(2020, 2031):
        ultima_prevalencia *= factor_crecimiento  # Aplicar el factor de crecimiento
        datos_con_estimaciones.append({
            'year': year,
            'nivel de ingresos': nivel,
            'prevalencia (%)': ultima_prevalencia
        })

# Convertir los resultados a un DataFrame
data_nivelingresos = pd.DataFrame(datos_con_estimaciones) # Data estimada hasta el 2030

# Asegurarse de que los datos de 'year' sean numéricos
data_nivelingresos['year'] = pd.to_numeric(data_nivelingresos['year'], errors='coerce')
data_nivelingresos = data_nivelingresos.dropna(subset=['year', 'prevalencia (%)'])  # Eliminar filas con datos faltantes
data_nivelingresos['year'] = data_nivelingresos['year'].astype(int)

# Crear el gráfico interactivo con Plotly
fig = px.line(data_nivelingresos, x='year', y='prevalencia (%)', color='nivel de ingresos',
              markers=True, title='Prevalencia histórica de anemia por nivel de ingresos',
              labels={'prevalencia (%)': 'Prevalencia (%)', 'year': 'Año'})

# Eliminar la leyenda
fig.update_layout(showlegend=False)

# Añadir el nombre del nivel de ingresos cerca del último punto disponible y alineado verticalmente
y_offset = 0.1  # Ajuste de distancia entre los nombres de los niveles
for i, level in enumerate(data_nivelingresos['nivel de ingresos'].unique()):
    level_data = data_nivelingresos[data_nivelingresos['nivel de ingresos'] == level]
    last_row = level_data[level_data['year'] == level_data['year'].max()]
    if not last_row.empty:
        last_year = last_row['year'].values[0]
        last_value = last_row['prevalencia (%)'].values[0]
        
        # Añadir la anotación en la posición deseada
        fig.add_annotation(
            x=last_year + 0.4,  # Posición horizontal cerca del último punto
            y=last_value + (y_offset * i),  # Posición vertical con ajuste para separar los nombres
            text=level,  # Texto con el nombre del nivel de ingresos
            showarrow=False,  # Sin flecha
            font=dict(size=10),  # Tamaño de la fuente
            xanchor='left',  # Alineación del texto a la izquierda
            align='left',  # Alineación del texto a la izquierda
        )

# Mejorar el diseño del gráfico
fig.update_traces(mode='lines+markers', hovertemplate='Prevalencia: %{y}')  # Solo mostrar prevalencia en el tooltip
fig.update_layout(
    xaxis_title='Año',
    yaxis_title='Prevalencia (%)',
    xaxis=dict(
        title=None,  # Quitar el título del eje X
        tickmode='array', 
        tickvals=sorted(
            data_nivelingresos['year'].unique()),
        showline=True,
        linecolor='black',
        ticks='outside',  # Mostrar marcas de graduación principales hacia el exterior
        tickwidth=1,  # Grosor de las marcas de graduación
    ),
    yaxis=dict(
            showline=True,  # Mostrar la línea del eje Y
            linewidth=1,  # Definir el grosor de la línea
            linecolor='black'  # Definir el color de la línea
    ),
    xaxis_tickangle=-90,  # Girar el eje de los años en sentido opuesto
    plot_bgcolor='white',
    font=dict(size=12),
    title_x=0.5,  # Centrar el título
)

# Mostrar el gráfico
fig.show()

#### 5.4.3. Predicción de la prevalencia de anemia infantil por país (Multiselección)

In [ ]:
# Cargar datos desde un archivo CSV llamado 'world_bank_anemia_paises_listo.csv'
data = pd.read_csv(r"world_bank_anemia_paises_listo.csv")

# Limpiar nombres de columnas (por si tienen espacios adicionales)
data.columns = data.columns.str.strip()

# Lista para almacenar los datos originales y las estimaciones
datos_con_estimaciones = []

# Obtener la lista de países únicos
paises_unicos = data['pais'].unique()

for pais in paises_unicos:
    # Filtrar los datos para el país actual
    datos_pais = data[data['pais'] == pais].sort_values(by='year')
    
    # Calcular las variaciones anuales porcentuales
    datos_pais['variacion'] = datos_pais['prevalencia (%)'].pct_change()
    
    # Calcular el promedio de la variación porcentual (ignorando valores nulos)
    factor_crecimiento = datos_pais['variacion'].mean() + 1  # Agregar 1 para obtener el factor multiplicativo
    
    # Agregar los datos originales del país al conjunto de datos
    for _, row in datos_pais.iterrows():
        datos_con_estimaciones.append({
            'year': row['year'],
            'pais': row['pais'],
            'prevalencia (%)': row['prevalencia (%)']
        })
    
    # Proyectar valores desde 2020 hasta 2030 usando el factor de crecimiento
    ultima_prevalencia = datos_pais['prevalencia (%)'].iloc[-1]  # Último valor conocido (2019)
    for year in range(2020, 2031):
        ultima_prevalencia *= factor_crecimiento  # Aplicar el factor de crecimiento
        datos_con_estimaciones.append({
            'year': year,
            'pais': pais,
            'prevalencia (%)': ultima_prevalencia
        })

# Convertir los resultados a un DataFrame
data_historico_pais_est = pd.DataFrame(datos_con_estimaciones) #Data con estimación hasta el 2030

# Transformar la variable 'year' a entero
data_historico_pais_est['year'] = pd.to_numeric(data_historico_pais_est['year'], errors='coerce')
data_historico_pais_est['year'] = data_historico_pais_est['year'].astype(int)

# Obtener la lista de países únicos
country_data = sorted(data_historico_pais_est['pais'].unique())

# Generar colores aleatorios para cada país
colors = {country: f"#{random.randint(0, 0xFFFFFF):06x}" for country in country_data}

# Función para completar los años faltantes
def completar_anios(df, country):
    country_data = df[df['pais'] == country]
    all_years = pd.DataFrame({'year': range(df['year'].min(), df['year'].max() + 1)})
    completed_data = pd.merge(all_years, country_data, on='year', how='left')
    completed_data['prevalencia (%)'] = completed_data['prevalencia (%)'].interpolate()
    completed_data['pais'] = completed_data['pais'].fillna(country)
    return completed_data

# Función para obtener estadísticas y generar mensajes
def obtener_estadisticas_mensaje(country_df):
    # Calcular el promedio histórico entre 2000 y 2019
    historical_data = country_df[(country_df['year'] >= 2000) & (country_df['year'] <= 2019)]
    avg_prevalence_2000_2019 = historical_data['prevalencia (%)'].mean()

    # Calcular la tasa de disminución promedio anual hasta 2030
    future_data = country_df[(country_df['year'] > 2019) & (country_df['year'] <= 2030)]
    if len(future_data) > 1:
        slope, _, _, _, _ = linregress(future_data['year'], future_data['prevalencia (%)'])
        annual_decrease_rate = -slope
    else:
        annual_decrease_rate = 0

    mensaje = (
        f"Para {country_df['pais'].iloc[0]}, la prevalencia de anemia tuvo un promedio de "
        f"{avg_prevalence_2000_2019:.2f}% entre 2000 y 2019. "
        f"Con base en las proyecciones, se estima que la prevalencia disminuirá a una tasa promedio anual de "
        f"{annual_decrease_rate:.2f}% hacia el año 2030."
    )
    return mensaje

# Función para graficar prevalencias históricas y mostrar estadísticas
def plot_selected_countries_plotly(countries_selected):
    if not countries_selected:
        with output_area:
            clear_output(wait=True)
            print("Por favor selecciona al menos un país.")
        return

    fig = go.Figure()
    mensajes = []

    for country in countries_selected:
        country_data = completar_anios(data_historico_pais_est, country)
        mensaje = obtener_estadisticas_mensaje(country_data)
        mensajes.append(mensaje)

        before_2020 = country_data[country_data['year'] < 2020]
        between_2020_2030 = country_data[(country_data['year'] >= 2020) & (country_data['year'] <= 2030)]
        after_2030 = country_data[country_data['year'] > 2030]

        country_color = colors[country]

        # Segmento antes de 2020
        fig.add_trace(go.Scatter(
            x=before_2020['year'], y=before_2020['prevalencia (%)'],
            mode='lines+markers', name=country,
            hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",
            line=dict(color=country_color)
        ))

        # Segmento entre 2020 y 2030
        fig.add_trace(go.Scatter(
            x=between_2020_2030['year'], y=between_2020_2030['prevalencia (%)'],
            mode='lines+markers', name=country,
            hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",
            line=dict(dash='dot', color=country_color)
        ))

        # Segmento después de 2030
        fig.add_trace(go.Scatter(
            x=after_2030['year'], y=after_2030['prevalencia (%)'],
            mode='lines+markers', name=country,
            hovertemplate="Prevalencia: %{y:.2f}<extra></extra>",
            line=dict(color=country_color)
        ))
        
        # Colocar el nombre del país ligeramente desplazado a la derecha de 2030
        year_2030_data = country_data[country_data['year'] == 2030]
        if not year_2030_data.empty:
            # Obtenemos el valor de prevalencia para 2030
            prev_2030 = year_2030_data['prevalencia (%)'].values[0]
            fig.add_annotation(
                x=2030 + 1,  # Desplazamos un poco a la derecha de 2030
                y=prev_2030,
                text=country,
                showarrow=False,
                font=dict(size=10, color='black'),
                xanchor='left',  # Alineación del texto a la izquierda
                align='left'  # Alineación del texto a la izquierda
            )

    with output_area:
        clear_output(wait=True)
        fig.update_layout(
            title={'text': 'Prevalencia histórica de anemia', 'x': 0.5, 'xanchor': 'center'},
            xaxis=dict(title=None, tickangle=-90, showline=True, linecolor='black', ticks='outside', tickwidth=1),
            yaxis=dict(showline=True, linewidth=1, linecolor='black', title='Prevalencia (%)'),
            template='plotly_white', width=850, showlegend=False
        )
        fig.show()
        for mensaje in mensajes:
            print(mensaje)

# Función para actualizar el gráfico dinámicamente
def update_plot(change=None):
    selected_countries = [cb.description for cb in checkboxes if cb.value]
    plot_selected_countries_plotly(selected_countries)

# Crear checkboxes para cada país
checkboxes = [widgets.Checkbox(value=False, description=country) for country in country_data]
for cb in checkboxes:
    cb.observe(update_plot, names='value')

# Crear contenedor de checkboxes y área de salida
checkbox_box = widgets.VBox(checkboxes, layout=widgets.Layout(overflow='auto', height='300px', width='200px'))
output_area = widgets.Output()

# Mostrar checkboxes y gráfico
display(widgets.HBox([checkbox_box, output_area]))

### 5.5 **CUARTA PARTE = ANALISIS DE LOS FACTORES QUE MÁS PUEDEN AFECTAR A LA ANEMIA INFANTIL (CASO EXCLUSIVO DE NIGERIA)**

In [ ]:
# Fuente: encuesta Nigeria
## Transformación de las variables a categóricas

# Cargar el archivo CSV datos_limpios_transformados.csv
import pandas as pd
data = pd.read_csv(r"datos_limpios_transformados.csv", sep=';')

# Tratar la variable 'Smokes' como categórica
data['Smokes'] = data['Smokes'].map({0: 'No', 1: 'Sí'})

# Tratar la variable 'Anemia_Level' como categórica 
anemia_mapping = {0: 'Medio', 1: 'Moderado', 2: 'No anémico', 3: 'Severo'}
data['Anemia_Level'] = data['Anemia_Level'].map(anemia_mapping)

# Tratar la variable 'Wealth_Index' como categórica con las nuevas categorías
wealth_mapping = {
    0: 'Medio',
    1: 'Pobre',
    2: 'Pobreza extrema',
    3: 'Rico',
    4: 'Riqueza alta'
}
data['Wealth_Index'] = data['Wealth_Index'].map(wealth_mapping)


# Tratar la variable 'Iron_Supplements' como categórica
data['Iron_Supplements'] = data['Iron_Supplements'].map({0: 'No sabe', 1: 'No', 2: 'Si'})

# Tratar la variable 'Iron_Supplements' como categórica
data['Residence_Type'] = data['Residence_Type'].map({0: 'Rural', 1: 'Urbana'})

# Verificar los cambios
# print(data[['Smokes', 'Anemia_Level', 'Wealth_Index', 'Iron_Supplements','Residence_Type']].head()) #para verificar

## 5.5.1 Nivel de anemia infantil según nivel de riqueza

In [ ]:
import plotly.express as px # Para hacer graficos interactivos
import pandas as pd # Para cargar, procesar y analizar datos en estructuras como DataFrame.

# Contar las observaciones para cada combinación de 'Anemia_Level' y 'Wealth_Index'
contado = data.groupby(['Anemia_Level', 'Wealth_Index']).size().reset_index(name='Count')

# Calcular el total por cada categoría de 'Wealth_Index'
contado['Total_Wealth_Index'] = contado.groupby('Wealth_Index')['Count'].transform('sum')

# Calcular el porcentaje dentro de cada 'Wealth_Index'
contado['Percentage'] = (contado['Count'] / contado['Total_Wealth_Index']) * 100

# Redondear los porcentajes a un solo decimal
contado['Percentage'] = contado['Percentage'].round(1)

# Definir el orden específico para 'Wealth_Index'
orden_wealth = ['Pobreza extrema', 'Pobre', 'Medio', 'Rico', 'Riqueza alta']

# Definir el orden específico para 'Anemia_Level'
orden_anemia = ['No anémico', 'Moderado', 'Medio', 'Severo']

# Convertir 'Wealth_Index' y 'Anemia_Level' en variables categóricas con el orden especificado
contado['Wealth_Index'] = pd.Categorical(contado['Wealth_Index'], categories=orden_wealth, ordered=True)
contado['Anemia_Level'] = pd.Categorical(contado['Anemia_Level'], categories=orden_anemia, ordered=True)

# Asegurarse de que los datos estén ordenados según las categorías especificadas
contado = contado.sort_values(by=['Wealth_Index', 'Anemia_Level'])

# Crear un gráfico de barras apiladas horizontales
fig = px.bar(contado, 
             x='Percentage', 
             y='Wealth_Index', 
             color='Anemia_Level', 
             orientation='h', 
             title='Nivel de anemia para cada nivel de riqueza',
             labels={'Percentage': 'Porcentaje (%)', 'Wealth_Index': 'Wealth Index'},
             hover_data={'Percentage': True, 'Anemia_Level': False, 'Wealth_Index': False})  # Mostrar solo el porcentaje en el tooltip

# Personalizar el layout
fig.update_layout(
    xaxis_title='Porcentaje (%)',
    yaxis_title='',
    barmode='stack',  # Apilar las barras
    bargap=0.15,  # Separación entre barras
    plot_bgcolor='white',  # Fondo blanco para el gráfico
    legend_title="Niveles de anemia"  # Cambiar el título de la leyenda
)

# Mostrar el gráfico
fig.show()

## 5.5.2 Nivel de anemia infantil según el consumo de suplementos de hierro

In [ ]:
import plotly.express as px # facilita la creación de widgets interactivos que se actualizan dinámicamente en función de las entradas del usuario.
from plotly.subplots import make_subplots # para poder poner subgraficos categorizados.

# Filtrar los datos según el valor de Iron_Supplements
data_yes = data[data['Iron_Supplements'] == 'Si']
data_no = data[data['Iron_Supplements'] == 'No']

# Contar la frecuencia de cada categoría de Anemia_Level
counts_yes = data_yes['Anemia_Level'].value_counts().reset_index()
counts_yes.columns = ['Anemia_Level', 'Count']

counts_no = data_no['Anemia_Level'].value_counts().reset_index()
counts_no.columns = ['Anemia_Level', 'Count']

# Gráfico para Iron_Supplements = Sí
fig_yes = px.pie(
    counts_yes,
    names='Anemia_Level',
    values='Count',
    color_discrete_sequence=px.colors.qualitative.Plotly,
    custom_data=['Count']  # Datos para el tooltip personalizado
)

fig_yes.update_traces(
    hovertemplate="N°: %{customdata[0]}<br>Nivel de anemia: %{label}<extra></extra>",
    pull=[0.05] * len(counts_yes)  # Espaciado entre segmentos
)

# Gráfico para Iron_Supplements = No
fig_no = px.pie(
    counts_no,
    names='Anemia_Level',
    values='Count',
    color_discrete_sequence=px.colors.qualitative.Plotly,
    custom_data=['Count']  # Datos para el tooltip personalizado
)

fig_no.update_traces(
    hovertemplate="N°: %{customdata[0]}<br>Nivel de anemia: %{label}<extra></extra>",
    pull=[0.05] * len(counts_no)  # Espaciado entre segmentos
)

# Crear subplots para mostrar los gráficos lado a lado
fig = make_subplots(
    rows=1, cols=2, 
    specs=[[{'type': 'domain'}, {'type': 'domain'}]], 
    subplot_titles=(
        "Distribución de Anemia_Level (Iron_Supplements = Sí)", 
        "Distribución de Anemia_Level (Iron_Supplements = No)"
    )
)

# Añadir el gráfico de "Sí"
fig.add_traces(fig_yes.data, rows=1, cols=1)

# Añadir el gráfico de "No"
fig.add_traces(fig_no.data, rows=1, cols=2)

# Configurar el diseño general
fig.update_layout(
    showlegend=True,  # Mantener una única leyenda
    legend_title="Niveles de Anemia",
    legend=dict(x=0.5, y=-0.2, orientation="h", xanchor="center"),  # Leyenda centrada debajo
    annotations=[  # Mover los títulos directamente sobre los gráficos
        dict(text="Niveles de anemia en consumidores de hierro", x=0.18, y=1.1, font_size=14, showarrow=False),
        dict(text="Niveles de anemia en NO consumidores de hierro", x=0.82, y=1.1, font_size=14, showarrow=False)
    ],
    margin=dict(t=50)  # Ajustar espacio superior
)

# Mostrar el gráfico
fig.show()

## 5.5.3 Nivel de anemia infantil según el tipo de residencia 

In [ ]:
import pandas as pd # Para el análisis y manipulación de datos en Python.
import plotly.express as px # Para crear visualizaciones interactivas.

# Contar las observaciones por combinación de 'Anemia_Level' y 'Residence_Type', especificando 'observed=False'
data_count_res = data.groupby(['Anemia_Level', 'Residence_Type'], observed=False).size().reset_index(name='count')

# Modificar los valores de 'count' a negativos cuando 'Residence_Type' sea 'Rural'
data_count_res['count'] = data_count_res.apply(lambda row: -row['count'] if row['Residence_Type'] == 'Rural' else row['count'], axis=1)

# Calcular el porcentaje tomando el valor absoluto de 'count'
total_per_anemia = data_count_res.groupby('Anemia_Level', observed=False)['count'].transform(lambda x: x.abs().sum())
data_count_res['percentage'] = (data_count_res['count'].abs() / total_per_anemia) * 100

# Crear el gráfico de barras horizontales apiladas con Plotly y cambiar la paleta de colores
fig = px.bar(
    data_count_res, 
    x='count', # Datos para el eje X
    y='Anemia_Level', # Datos para el eje Y
    color='Residence_Type', # Colorea las barras según el tipo de residencia.
    orientation='h', # Especifica que las barras deben ser horizontales.
    title='Nivel de anemia según el tipo de residencia', # Define el título del gráfico.
    labels={'count': 'Número de Observaciones', 'Anemia_Level': 'Nivel de Anemia', 'Residence_Type': 'Tipo de Residencia'}, #Renombra las columnas del eje y la leyenda.
    hover_data={'percentage': ':.1f%', 'count': True}, 
    color_discrete_sequence=["#1f77b4", "#ff7f0e"],  # Paleta de colores personalizada
    category_orders={"Anemia_Level": ["Severo", "Medio", "Moderado", "No anémico"]}  # Ordena los niveles de anemia de la manera especificada.
)

# Personalizar el tooltip para mostrar el número absoluto de observaciones y el porcentaje
fig.update_traces(
    hovertemplate="<b>N°:</b> %{customdata[0]:.0f} <br><b>Porcent.:</b> %{customdata[0]:.1f}%<extra></extra>"
)

# Ocultar los valores del eje X colocando el color del texto en blanco
fig.update_layout(
    plot_bgcolor='white',  # Fondo blanco del gráfico
    xaxis=dict(
        showticklabels=False,  # Ocultar los valores del eje X
        title=None,  # Título del eje X
        linecolor='black',  # Color de la línea del eje X
        linewidth=1  # Grosor de la línea del eje X
    ),
    yaxis=dict(
        tickfont=dict(color='black'),  # Cambiar el color de las etiquetas del eje Y a negro
        linecolor='black',  # Color de la línea del eje Y
        linewidth=1  # Grosor de la línea del eje Y
    ),
    title=dict(font=dict(size=18, color='black')),  # Color del título
    template="simple_white"  # Estilo de plantilla blanca
)

# Mostrar el gráfico
fig.show()